# Install Dependencies
Install additional packages needed to run SQL command in Jupyter cells and send them to a Postgres database server.

## PostgreSQL
Allows for a self-contained instance of PostresSQL for this notebook.

In [ ]:
%mamba install -y -c conda-forge postgresql=17 --quiet
!psql --version

## PostgreSQL Adapter for Python
Essential for connecting and interacting with PostgreSQL databases from Python code in Jupyter notebooks.

In [ ]:
%mamba install -c conda-forge psycopg2 -y --quiet
!python -c "import psycopg2; print(psycopg2.__version__)"

## Language Server Protocol for Jupyter
Enables code completion, hover documentation, and real-time syntax checking for multiple languages in Jupyter notebooks, including SQL.

In [ ]:
%mamba install -c conda-forge jupyterlab-lsp -y --quiet
!jupyter labextension list
!jupyter server extension list

## Language Server for Python
Provides real-time linting, code suggestions, and auto-completion in Jupyter notebooks for Python code.

In [ ]:
%mamba install -c conda-forge python-lsp-server -y --quiet
!pylsp --version

## SQL Magic Extension for Jupyter
Allows running SQL queries directly in Jupyter notebooks using SQL magic (%%sql). Enables seamless execution of SQL queries inside a Jupyter Notebook without switching to a database client.

In [ ]:
%mamba install -c conda-forge jupysql -y --quiet
!python -c "import sql; print(sql.__version__)"
!jupyter labextension list

## SQL Editor Extension
Enables SQL-specific features in Jupyter notebooks, such as:
* Syntax highlighting
* Query auto-completion
* Error checking

In [ ]:
%pip install jupyterlab_sql_editor --quiet
!jupyter labextension list

# Start PostgreSQL

## Define paths and logfile

In [ ]:
import os

DB_DIR = "mylocal_db"
LOGFILE = "postgres_log.txt"
PG_HBA_PATH = os.path.join(DB_DIR, "pg_hba.conf")

## Check PostgreSQL status

In [ ]:
!pg_ctl -D $DB_DIR status

## Stop any existing PostgreSQL process

In [ ]:
!pg_ctl stop -D $DB_DIR -m fast || echo "Nothing to stop."

## Initialize the database if necessary

In [ ]:
import os

if not os.path.exists(DB_DIR) or not os.path.exists(os.path.join(DB_DIR, "PG_VERSION")):
    os.makedirs(DB_DIR, exist_ok=True)
    !initdb -D $DB_DIR

## Modify pg_hba.conf to allow trust authentication for user "notebook"

In [ ]:
with open(PG_HBA_PATH, "a") as pg_hba:
    pg_hba.write("\nlocal   all             notebook                                trust\n")
    pg_hba.write("host    all             notebook        127.0.0.1/32          trust\n")
    pg_hba.write("host    all             notebook        ::1/128               trust\n")

## Start PostgreSQL server

In [ ]:
import subprocess
import time
import atexit

def stop_postgres():
    !pg_ctl -D $DB_DIR stop

atexit.register(stop_postgres)

with open(LOGFILE, "a") as logfile:
    subprocess.Popen(["pg_ctl", "-D", DB_DIR, "-l", LOGFILE, "start"],
                     stdout=logfile, stderr=logfile)
time.sleep(5)

## Create the "notebook" user with full permissions

In [ ]:
!psql -d postgres -c "CREATE USER notebook WITH SUPERUSER;" || echo "User already exists."
!psql -d postgres -c "ALTER USER notebook WITH PASSWORD NULL;"

# Connect

## Load & Configure SQLMagic

In [ ]:
%load_ext sql
%config SqlMagic.displaylimit = None
%config SqlMagic.named_parameters = "enabled"

## Create Fresh sahuagin Database

In [ ]:
%sql postgresql://notebook@localhost:5432/postgres
%sql SELECT pg_terminate_backend(pg_stat_activity.pid) FROM pg_stat_activity WHERE pg_stat_activity.datname = 'sahuagin';
%sql DROP DATABASE IF EXISTS sahuagin;
%sql CREATE DATABASE sahuagin;

## Connect to sahuagin Database

In [ ]:
%sql postgresql://notebook@localhost:5432/sahuagin

# Create Tables & Types

## Attribute Type

In [ ]:
%%sql

DROP TYPE IF EXISTS attribute_type CASCADE;
CREATE TYPE attribute_type AS ENUM ('discrete','continuous');

## Attribute

In [ ]:
%%sql

DROP TABLE IF EXISTS attribute CASCADE;
CREATE TABLE attribute (
    id integer GENERATED ALWAYS AS IDENTITY PRIMARY KEY,
    name varchar(255) NOT NULL UNIQUE,
    type attribute_type NOT NULL,
    decimals integer,
    has_labels boolean,
    has_value boolean,
    max_value double precision,
    min_value double precision,
    mode_value double precision,
    concentration double precision,
    skew double precision,
    units varchar(255),
    CHECK (
        max_value IS NULL
        OR min_value IS NULL
        OR max_value >= min_value
    ),
    CHECK (
        (
            type = 'discrete'
            AND decimals IS NULL
            AND has_labels IS NULL
            AND has_value IS NULL
            AND max_value IS NULL
            AND min_value IS NULL
            AND mode_value IS NULL
            AND concentration IS NULL
            AND skew IS NULL
            AND units IS NULL
        )
        OR (
            type = 'continuous'
            AND decimals IS NOT NULL
            AND has_labels IS NOT NULL
            AND has_value IS NOT NULL
            AND max_value IS NOT NULL
            AND min_value IS NOT NULL
            AND mode_value IS NOT NULL
            AND concentration IS NOT NULL
            AND skew IS NOT NULL
        )
    )
);

## Span

In [ ]:
%%sql

DROP TABLE IF EXISTS span CASCADE;
CREATE TABLE span (
    id integer GENERATED ALWAYS AS IDENTITY PRIMARY KEY,
    attribute_id integer NOT NULL,
    label varchar(255) NOT NULL,
    type attribute_type NOT NULL,
    is_percentage_pinned boolean,
    weight double precision,
    max_value double precision,
    min_value double precision,
    CONSTRAINT unique_attribute_type_label UNIQUE (attribute_id, type, label),
    CONSTRAINT fk_spans_attr FOREIGN KEY (attribute_id) REFERENCES attribute (id) ON DELETE CASCADE,
    CHECK (
        max_value IS NULL
        OR min_value IS NULL
        OR max_value >= min_value
    ),
    CHECK (
        (
            type = 'discrete'
            AND is_percentage_pinned IS NOT NULL
            AND weight IS NOT NULL
            AND max_value IS NULL
            AND min_value IS NULL
        )
        OR (
            type = 'continuous'
            AND is_percentage_pinned IS NULL
            AND weight IS NULL
            AND max_value IS NOT NULL
            AND min_value IS NOT NULL
        )
    )
);

## Variant

In [ ]:
%%sql

DROP TABLE IF EXISTS variant CASCADE;
CREATE TABLE variant (
    id integer GENERATED ALWAYS AS IDENTITY PRIMARY KEY,
    name varchar(255) NOT NULL UNIQUE
);

## Attribute Reference

In [ ]:
%%sql

DROP TABLE IF EXISTS variant_attribute CASCADE;
CREATE TABLE variant_attribute (
    id integer GENERATED ALWAYS AS IDENTITY PRIMARY KEY,
    attribute_id integer NOT NULL,
    name varchar(255) NOT NULL,
    causation_index integer NOT NULL,
    variant_id integer NOT NULL,
    CONSTRAINT unique_variant_name UNIQUE (variant_id, name),
    CONSTRAINT unique_variant_index UNIQUE (variant_id, causation_index),
    CONSTRAINT fk_variant_attr_attr FOREIGN KEY (attribute_id) REFERENCES attribute (id) ON DELETE CASCADE,
    CONSTRAINT fk_variant_attr_variant FOREIGN KEY (variant_id) REFERENCES variant (id) ON DELETE CASCADE
);

## Variant Reference Span

In [ ]:
%%sql

DROP TABLE IF EXISTS variant_attr_span CASCADE;
CREATE TABLE variant_attr_span (
    id integer GENERATED ALWAYS AS IDENTITY PRIMARY KEY,
    span_id integer NOT NULL,
    variant_attribute_id integer NOT NULL,
    variant_id integer NOT NULL,
    CONSTRAINT unique_variant_attribute_span UNIQUE (variant_id, variant_attribute_id, span_id),
    CONSTRAINT fk_varattr_span_span FOREIGN KEY (span_id) REFERENCES span (id) ON DELETE CASCADE,
    CONSTRAINT fk_varattr_span_varattr FOREIGN KEY (variant_attribute_id) REFERENCES variant_attribute (id) ON DELETE CASCADE,
    CONSTRAINT fk_varattr_span_variant FOREIGN KEY (variant_id) REFERENCES variant (id) ON DELETE CASCADE
);

## Possible Attribute

In [ ]:
%%sql

DROP TABLE IF EXISTS vavspan_attr CASCADE;
CREATE TABLE vavspan_attr (
    id integer GENERATED ALWAYS AS IDENTITY PRIMARY KEY,
    variant_attribute_id integer NOT NULL,
    variant_attr_span_id integer,
    parent_vavspan_attr_id integer,
    CONSTRAINT unique_vavspan_attr UNIQUE (parent_vavspan_attr_id, variant_attr_span_id, variant_attribute_id),
    CONSTRAINT fk_vavspan_attr_va FOREIGN KEY (variant_attribute_id) REFERENCES variant_attribute (id) ON DELETE CASCADE,
    CONSTRAINT fk_vavspan_attr_vas FOREIGN KEY (variant_attr_span_id) REFERENCES variant_attr_span (id) ON DELETE CASCADE,
    CONSTRAINT fk_vavspan_attr_parent FOREIGN KEY (parent_vavspan_attr_id) REFERENCES vavspan_attr (id) ON DELETE CASCADE
);

## Causal Link

In [ ]:
%%sql

DROP TABLE IF EXISTS variation CASCADE;
CREATE TABLE variation (
    id integer GENERATED ALWAYS AS IDENTITY PRIMARY KEY,
    to_modify_vavspan_attr_id integer NOT NULL,
    activating_vavspan_attr_id integer NOT NULL,
    CONSTRAINT unique_span_to_attribute UNIQUE (activating_vavspan_attr_id, to_modify_vavspan_attr_id),
    CONSTRAINT fk_variation_vavspan_to_modify FOREIGN KEY (to_modify_vavspan_attr_id) REFERENCES vavspan_attr (id) ON DELETE CASCADE,
    CONSTRAINT fk_variation_vavspan_activating FOREIGN KEY (activating_vavspan_attr_id) REFERENCES vavspan_attr (id) ON DELETE CASCADE
);

## Effect

In [ ]:
%%sql

DROP TABLE IF EXISTS effect CASCADE;
CREATE TABLE effect (
    id integer GENERATED ALWAYS AS IDENTITY PRIMARY KEY,
    name varchar(255) NOT NULL,
    variant_id integer NOT NULL,
    variation_id integer NOT NULL,
    CONSTRAINT unique_effect_name_within_variant UNIQUE (name, variant_id),
    CONSTRAINT fk_effect_variant FOREIGN KEY (variant_id) REFERENCES variant (id) ON DELETE CASCADE,
    CONSTRAINT fk_effect_variation FOREIGN KEY (variation_id) REFERENCES variation (id) ON DELETE CASCADE
);

## Delta Parameter Effect

In [ ]:
%%sql

DROP TABLE IF EXISTS variation_continuous_attr CASCADE;
CREATE TABLE variation_continuous_attr (    
    id integer GENERATED ALWAYS AS IDENTITY PRIMARY KEY,
    effect_id integer NOT NULL UNIQUE,
    delta_mode double precision NOT NULL,
    delta_conc double precision NOT NULL,
    delta_skew double precision NOT NULL,
    CONSTRAINT fk_var_continuous_attr_effect FOREIGN KEY (effect_id) REFERENCES effect (id) ON DELETE CASCADE
);

## Additional Possibility Effect

In [ ]:
%%sql

DROP TABLE IF EXISTS variation_activated_span CASCADE;
CREATE TABLE variation_activated_span (
    id integer GENERATED ALWAYS AS IDENTITY PRIMARY KEY,
    effect_id integer NOT NULL UNIQUE,
    span_id integer NOT NULL,
    CONSTRAINT fk_var_activated_span_span FOREIGN KEY (span_id) REFERENCES span (id) ON DELETE CASCADE,
    CONSTRAINT fk_var_activated_span_effect FOREIGN KEY (effect_id) REFERENCES effect (id) ON DELETE CASCADE
);

## Delta Weight Effect

In [ ]:
%%sql

DROP TABLE IF EXISTS variation_delta_weight CASCADE;
CREATE TABLE variation_delta_weight (
    id integer GENERATED ALWAYS AS IDENTITY PRIMARY KEY,
    effect_id integer NOT NULL UNIQUE,
    span_id integer NOT NULL,
    delta_weight double precision NOT NULL,
    CONSTRAINT fk_var_delta_weight_span FOREIGN KEY (span_id) REFERENCES span (id) ON DELETE CASCADE,
    CONSTRAINT fk_var_delta_weight_effect FOREIGN KEY (effect_id) REFERENCES effect (id) ON DELETE CASCADE
);

## Removed Possibility Effect

In [ ]:
%%sql

DROP TABLE IF EXISTS variation_inactive_span CASCADE;
CREATE TABLE variation_inactive_span (
    id integer GENERATED ALWAYS AS IDENTITY PRIMARY KEY,
    effect_id integer NOT NULL UNIQUE,
    span_id integer NOT NULL,
    CONSTRAINT fk_var_inactive_span_span FOREIGN KEY (span_id) REFERENCES span (id) ON DELETE CASCADE,
    CONSTRAINT fk_var_inactive_span_effect FOREIGN KEY (effect_id) REFERENCES effect (id) ON DELETE CASCADE
);

## Entity

In [ ]:
%%sql

DROP TABLE IF EXISTS entity CASCADE;
CREATE TABLE entity (
    id integer GENERATED ALWAYS AS IDENTITY PRIMARY KEY,
    variant_id integer NOT NULL,
    name varchar(255) NOT NULL UNIQUE,
    CONSTRAINT fk_entity_variant FOREIGN KEY (variant_id) REFERENCES variant (id) ON DELETE CASCADE
);

## Entity Group

In [ ]:
%%sql

DROP TABLE IF EXISTS entity_group CASCADE;
CREATE TABLE entity_group (
    id integer GENERATED ALWAYS AS IDENTITY PRIMARY KEY,
    name varchar(255) NOT NULL UNIQUE
);

## Entity Group Link

In [ ]:
%%sql

DROP TABLE IF EXISTS entity_group_link CASCADE;
CREATE TABLE entity_group_link (
    id integer GENERATED ALWAYS AS IDENTITY PRIMARY KEY,
    entity_id integer NOT NULL,
    entity_group_id integer NOT NULL,
    CONSTRAINT unique_link_entity_group UNIQUE (entity_group_id, entity_id),
    CONSTRAINT fk_entity FOREIGN KEY (entity_id) REFERENCES entity (id) ON DELETE CASCADE,
    CONSTRAINT fk_entity_group FOREIGN KEY (entity_group_id) REFERENCES entity_group (id) ON DELETE CASCADE
);

## Entity State

In [ ]:
%%sql

DROP TABLE IF EXISTS entity_state CASCADE;
CREATE TABLE entity_state (
    id integer GENERATED ALWAYS AS IDENTITY PRIMARY KEY,
    entity_id integer NOT NULL,
    time double precision NOT NULL,
    CONSTRAINT unique_entity_time UNIQUE (entity_id, time),
    CONSTRAINT fk_entity FOREIGN KEY (entity_id) REFERENCES entity (id) ON DELETE CASCADE
);

## Attribute Value

In [ ]:
%%sql

DROP TABLE IF EXISTS entity_varattr_value CASCADE;
CREATE TABLE entity_varattr_value (
    id integer GENERATED ALWAYS AS IDENTITY PRIMARY KEY,
    entity_state_id integer NOT NULL,
    numeric_value double precision,
    span_id integer,
    vavspan_attr_id integer NOT NULL,
    CONSTRAINT unique_state_vavspan_attr UNIQUE (entity_state_id, vavspan_attr_id),
    CONSTRAINT fk_evav_entity_state FOREIGN KEY (entity_state_id) REFERENCES entity_state (id) ON DELETE CASCADE,
    CONSTRAINT fk_evav_span FOREIGN KEY (span_id) REFERENCES span (id) ON DELETE CASCADE,
    CONSTRAINT fk_evav_vavspan_attr FOREIGN KEY (vavspan_attr_id) REFERENCES vavspan_attr (id) ON DELETE CASCADE
);

## Attribute Value Lock

In [ ]:
%%sql

DROP TABLE IF EXISTS evav_lock CASCADE;
CREATE TABLE evav_lock (
    id integer GENERATED ALWAYS AS IDENTITY PRIMARY KEY,
    locked_evav_id integer NOT NULL,
    locking_evav_id integer,
    CONSTRAINT unique_evav_lock UNIQUE (locking_evav_id, locked_evav_id),
    CONSTRAINT fk_evav_lock_locked_evav FOREIGN KEY (locked_evav_id) REFERENCES entity_varattr_value (id) ON DELETE CASCADE,
    CONSTRAINT fk_evav_lock_locking_evav FOREIGN KEY (locking_evav_id) REFERENCES entity_varattr_value (id) ON DELETE CASCADE
);

# Create Indexes

## idx_attribute_name

In [397]:
%%sql

DROP INDEX IF EXISTS idx_attribute_name;
CREATE INDEX idx_attribute_name ON attribute (name);

Running query in 'postgresql://notebook@localhost:5432/sahuagin'

++
||
++
++

## idx_span_attribute_id

In [398]:
%%sql

DROP INDEX IF EXISTS idx_span_attribute_id;
CREATE INDEX idx_span_attribute_id ON span (attribute_id);

Running query in 'postgresql://notebook@localhost:5432/sahuagin'

++
||
++
++

## idx_span_attribute_type

In [399]:
%%sql

DROP INDEX IF EXISTS idx_span_attribute_type;
CREATE INDEX idx_span_attribute_type ON span(attribute_id, type, id);

Running query in 'postgresql://notebook@localhost:5432/sahuagin'

++
||
++
++

## idx_span_attr_type

In [400]:
%%sql

DROP INDEX IF EXISTS idx_span_attr_type;
CREATE INDEX idx_span_attr_type ON span(attribute_id, type, min_value, max_value);

Running query in 'postgresql://notebook@localhost:5432/sahuagin'

++
||
++
++

## idx_span_attr_type_pinned

In [401]:
%%sql

DROP INDEX IF EXISTS idx_span_attr_type_pinned;
CREATE INDEX idx_span_attr_type_pinned ON span(attribute_id, type, is_percentage_pinned);

Running query in 'postgresql://notebook@localhost:5432/sahuagin'

++
||
++
++

## idx_span_attr_type_pinned_wl

In [402]:
%%sql

DROP INDEX IF EXISTS idx_span_attr_type_pinned_wl;
CREATE INDEX idx_span_attr_type_pinned_wl ON span(attribute_id, type, is_percentage_pinned, weight, label);

Running query in 'postgresql://notebook@localhost:5432/sahuagin'

++
||
++
++

## idx_variant_name

In [403]:
%%sql

DROP INDEX IF EXISTS idx_variant_name;
CREATE INDEX idx_variant_name ON variant (name);

Running query in 'postgresql://notebook@localhost:5432/sahuagin'

++
||
++
++

## idx_variant_attribute_attribute

In [404]:
%%sql

DROP INDEX IF EXISTS idx_variant_attribute_attribute;
CREATE INDEX idx_variant_attribute_attribute ON variant_attribute (attribute_id);

Running query in 'postgresql://notebook@localhost:5432/sahuagin'

++
||
++
++

## idx_variant_attribute_variant

In [405]:
%%sql

DROP INDEX IF EXISTS idx_variant_attribute_variant;
CREATE INDEX idx_variant_attribute_variant ON variant_attribute (variant_id);

Running query in 'postgresql://notebook@localhost:5432/sahuagin'

++
||
++
++

## idx_variant_attribute_variant_causation

In [406]:
%%sql

DROP INDEX IF EXISTS idx_variant_attribute_variant_causation;
CREATE INDEX idx_variant_attribute_variant_causation ON variant_attribute (variant_id, causation_index);

Running query in 'postgresql://notebook@localhost:5432/sahuagin'

++
||
++
++

## idx_variant_attr_span_span_id

In [407]:
%%sql

DROP INDEX IF EXISTS idx_variant_attr_span_span_id;
CREATE INDEX idx_variant_attr_span_span_id ON variant_attr_span (span_id);

Running query in 'postgresql://notebook@localhost:5432/sahuagin'

++
||
++
++

## idx_variant_attr_span_variant_attribute_id

In [408]:
%%sql

DROP INDEX IF EXISTS idx_variant_attr_span_variant_attribute_id;
CREATE INDEX idx_variant_attr_span_variant_attribute_id ON variant_attr_span (variant_attribute_id);

Running query in 'postgresql://notebook@localhost:5432/sahuagin'

++
||
++
++

## idx_variant_attr_span_variant_attr

In [409]:
%%sql

DROP INDEX IF EXISTS idx_variant_attr_span_variant_attr;
CREATE INDEX idx_variant_attr_span_variant_attr ON variant_attr_span (
    variant_id,
    variant_attribute_id
);

Running query in 'postgresql://notebook@localhost:5432/sahuagin'

++
||
++
++

## idx_variant_attr_span_vaid_spanid

In [410]:
%%sql

DROP INDEX IF EXISTS idx_variant_attr_span_vaid_spanid;
CREATE INDEX idx_variant_attr_span_vaid_spanid ON variant_attr_span (variant_attribute_id, span_id);

Running query in 'postgresql://notebook@localhost:5432/sahuagin'

++
||
++
++

## idx_variant_attr_span_vaid_variant

In [411]:
%%sql

DROP INDEX IF EXISTS idx_variant_attr_span_vaid_variant;
CREATE INDEX idx_variant_attr_span_vaid_variant ON variant_attr_span (
    variant_attribute_id,
    variant_id
);

Running query in 'postgresql://notebook@localhost:5432/sahuagin'

++
||
++
++

## idx_vavspan_attr_variant_attribute

In [412]:
%%sql

DROP INDEX IF EXISTS idx_vavspan_attr_variant_attribute;
CREATE INDEX idx_vavspan_attr_variant_attribute ON vavspan_attr (variant_attribute_id);

Running query in 'postgresql://notebook@localhost:5432/sahuagin'

++
||
++
++

## idx_vavspan_attr_vaid

In [413]:
%%sql

DROP INDEX IF EXISTS idx_vavspan_attr_vaid;
CREATE INDEX idx_vavspan_attr_vaid ON vavspan_attr (variant_attribute_id, id);

Running query in 'postgresql://notebook@localhost:5432/sahuagin'

++
||
++
++

## idx_vavspan_attr_parent

In [414]:
%%sql

DROP INDEX IF EXISTS idx_vavspan_attr_parent;
CREATE INDEX idx_vavspan_attr_parent ON vavspan_attr(parent_vavspan_attr_id);

Running query in 'postgresql://notebook@localhost:5432/sahuagin'

++
||
++
++

## idx_uq_vavspan_attr

In [415]:
%%sql

DROP INDEX IF EXISTS idx_uq_vavspan_attr;
CREATE INDEX idx_uq_vavspan_attr ON vavspan_attr (
    variant_attribute_id,
    variant_attr_span_id
);

Running query in 'postgresql://notebook@localhost:5432/sahuagin'

++
||
++
++

## idx_var_cont_variation

In [416]:
%%sql

DROP INDEX IF EXISTS idx_var_cont_variation;
CREATE INDEX idx_var_cont_variation ON variation_continuous_attr (effect_id);

Running query in 'postgresql://notebook@localhost:5432/sahuagin'

++
||
++
++

## idx_variation_activating_vavspan

In [417]:
%%sql

DROP INDEX IF EXISTS idx_variation_activating_vavspan;
CREATE INDEX idx_variation_activating_vavspan ON variation(activating_vavspan_attr_id);

Running query in 'postgresql://notebook@localhost:5432/sahuagin'

++
||
++
++

## idx_variation_to_modify_vavspan

In [418]:
%%sql

DROP INDEX IF EXISTS idx_variation_to_modify_vavspan;
CREATE INDEX idx_variation_to_modify_vavspan ON variation(to_modify_vavspan_attr_id);

Running query in 'postgresql://notebook@localhost:5432/sahuagin'

++
||
++
++

## idx_variation_activated_span_varid_spanid

In [419]:
%%sql

DROP INDEX IF EXISTS idx_variation_activated_span_effid_spanid;
CREATE INDEX idx_variation_activated_span_effid_spanid ON variation_activated_span (effect_id, span_id);

Running query in 'postgresql://notebook@localhost:5432/sahuagin'

++
||
++
++

## idx_variation_inactive

In [420]:
%%sql

DROP INDEX IF EXISTS idx_variation_inactive;
CREATE INDEX idx_variation_inactive ON variation_inactive_span (effect_id, span_id);

Running query in 'postgresql://notebook@localhost:5432/sahuagin'

++
||
++
++

## idx_variation_delta_weight_varid_spanid

In [421]:
%%sql

DROP INDEX IF EXISTS idx_variation_delta_weight_effid_spanid;
CREATE INDEX idx_variation_delta_weight_effid_spanid ON variation_delta_weight (effect_id, span_id);

Running query in 'postgresql://notebook@localhost:5432/sahuagin'

++
||
++
++

## idx_variation_delta

In [422]:
%%sql

DROP INDEX IF EXISTS idx_variation_delta;
CREATE INDEX idx_variation_delta ON variation_delta_weight (
    effect_id,
    span_id,
    delta_weight
);

Running query in 'postgresql://notebook@localhost:5432/sahuagin'

++
||
++
++

## idx_entity_state_entity_id

In [423]:
%%sql

DROP INDEX IF EXISTS idx_entity_state_entity_id;
CREATE INDEX idx_entity_state_entity_id ON entity_state (entity_id);

Running query in 'postgresql://notebook@localhost:5432/sahuagin'

++
||
++
++

## idx_entity_state_entity_time

In [424]:
%%sql

DROP INDEX IF EXISTS idx_entity_state_entity_time;
CREATE INDEX idx_entity_state_entity_time ON entity_state (entity_id, time);

Running query in 'postgresql://notebook@localhost:5432/sahuagin'

++
||
++
++

## idx_entity_varattr_value_entity_state_id

In [425]:
%%sql

DROP INDEX IF EXISTS idx_entity_varattr_value_entity_state_id;
CREATE INDEX idx_entity_varattr_value_entity_state_id ON entity_varattr_value (entity_state_id);

Running query in 'postgresql://notebook@localhost:5432/sahuagin'

++
||
++
++

## idx_evav_state_vaid

In [426]:
%%sql

DROP INDEX IF EXISTS idx_evav_state_vaid;
CREATE INDEX idx_evav_state_vaid ON entity_varattr_value (
    entity_state_id,
    vavspan_attr_id
);

Running query in 'postgresql://notebook@localhost:5432/sahuagin'

++
||
++
++

## idx_evav_state_attr_span

In [427]:
%%sql

DROP INDEX IF EXISTS idx_evav_state_attr_span;
CREATE INDEX idx_evav_state_attr_span ON entity_varattr_value (
    entity_state_id,
    vavspan_attr_id,
    span_id
);

Running query in 'postgresql://notebook@localhost:5432/sahuagin'

++
||
++
++

## idx_evav_lock_locked

In [428]:
%%sql

DROP INDEX IF EXISTS idx_evav_lock_locked;
CREATE INDEX idx_evav_lock_locked ON evav_lock (locked_evav_id);

Running query in 'postgresql://notebook@localhost:5432/sahuagin'

++
||
++
++

## idx_evav_lock_locking

In [429]:
%%sql

DROP INDEX IF EXISTS idx_evav_lock_locking;
CREATE INDEX idx_evav_lock_locking ON evav_lock (locking_evav_id);

Running query in 'postgresql://notebook@localhost:5432/sahuagin'

++
||
++
++

## idx_evav_lock_locked_locking

In [430]:
%%sql

DROP INDEX IF EXISTS idx_evav_lock_locked_locking;
CREATE INDEX idx_evav_lock_locked_locking ON evav_lock (
    locked_evav_id,
    locking_evav_id
);

Running query in 'postgresql://notebook@localhost:5432/sahuagin'

++
||
++
++

## idx_entity_group_link_group_id_entity_id

In [431]:
%%sql

DROP INDEX IF EXISTS idx_entity_group_link_group_id_entity_id;
CREATE INDEX idx_entity_group_link_group_id_entity_id
  ON entity_group_link(entity_group_id, entity_id);

Running query in 'postgresql://notebook@localhost:5432/sahuagin'

++
||
++
++

# Debug

## Create Log Table

In [ ]:
%%sql

DROP TABLE IF EXISTS debug_log CASCADE;
CREATE TABLE debug_log (
    id integer GENERATED ALWAYS AS IDENTITY PRIMARY KEY,
    log_time timestamp DEFAULT CURRENT_TIMESTAMP,
    procedure_name varchar(255),
    log_message text
);

## Create Log Function

In [ ]:
%%sql

CREATE OR REPLACE FUNCTION debug_log(
    p_procedure_name varchar,
    p_log_message text
) RETURNS void
LANGUAGE plpgsql
AS $$
BEGIN
    INSERT INTO debug_log (procedure_name, log_message, log_time)
    VALUES (p_procedure_name, p_log_message, now());
END;
$$;

## Clear Log Table

In [ ]:
%%sql

TRUNCATE TABLE debug_log RESTART IDENTITY CASCADE;

# Create Functions & Procedures

## Discreate Attribute

### Add

In [792]:
%%sql

CREATE OR REPLACE PROCEDURE add_discrete_attribute(
    in_name       VARCHAR(255),
    in_spans      JSON  -- JSON array: either [["span1", 10], ["span2", 20]] or ["span1", "span2"]
)
LANGUAGE plpgsql
AS $$
DECLARE
    new_attr_id       INTEGER;
    span_count        INTEGER;
    span_label        VARCHAR(255);
    span_weight       DOUBLE PRECISION;
    normalized_weight DOUBLE PRECISION;
    total_weight      DOUBLE PRECISION := 0;
    first_elem_type   TEXT;
    spans_json        JSON;
    i                 INTEGER;
BEGIN
    -- Create the new attribute
    INSERT INTO attribute (name, type)
      VALUES (in_name, 'discrete')
      RETURNING id INTO new_attr_id;

    IF in_spans IS NOT NULL THEN
        spans_json := in_spans::json;
        span_count := json_array_length(spans_json);

        IF span_count > 0 THEN
            first_elem_type := json_typeof(spans_json->0);
        ELSE
            first_elem_type := '';
        END IF;

        IF first_elem_type = 'array' THEN
            -- First pass: compute the total weight
            FOR i IN 0..span_count - 1 LOOP
                total_weight := total_weight + (((spans_json->i)->>1)::DOUBLE PRECISION);
            END LOOP;

            -- Second pass: insert spans with normalized weight
            FOR i IN 0..span_count - 1 LOOP
                span_label := (spans_json->i)->>0;
                span_weight := ((spans_json->i)->>1)::DOUBLE PRECISION;
                normalized_weight := span_weight / total_weight;
                INSERT INTO span (attribute_id, label, type, is_percentage_pinned, weight)
                  VALUES (new_attr_id, span_label, 'discrete', false, normalized_weight);
            END LOOP;
        ELSE
            -- For a JSON array of labels only, call the helper procedure for each span.
            FOR i IN 0..span_count - 1 LOOP
                span_label := spans_json->>i;
                CALL add_discrete_span(new_attr_id, span_label);
            END LOOP;
        END IF;
    END IF;
END;
$$;

Running query in 'postgresql://notebook@localhost:5432/sahuagin'

++
||
++
++

### Get

In [793]:
%%sql

CREATE OR REPLACE FUNCTION get_discrete_attribute(p_attribute_name varchar)
RETURNS TABLE (
    attribute   varchar,
    type        attribute_type,
    span_label  varchar,
    span_weight double precision,
    is_pinned      boolean
)
LANGUAGE plpgsql
AS
$$
BEGIN
  RETURN QUERY
    SELECT 
      a.name,
      a.type,
      s.label,
      s.weight,
      s.is_percentage_pinned
    FROM attribute a
    JOIN span s ON s.attribute_id = a.id
    WHERE a.name = p_attribute_name
      AND NOT EXISTS (
          SELECT 1 
          FROM variation_activated_span vas 
          WHERE vas.span_id = s.id
      );
END;
$$;

Running query in 'postgresql://notebook@localhost:5432/sahuagin'

++
||
++
++

### Delete

In [794]:
%%sql

CREATE OR REPLACE PROCEDURE delete_discrete_attribute(p_attribute_name varchar)
LANGUAGE plpgsql
AS
$$
BEGIN
  DELETE FROM attribute
  WHERE name = p_attribute_name
    AND type = 'discrete';
END;
$$;

Running query in 'postgresql://notebook@localhost:5432/sahuagin'

++
||
++
++

## Redistribute Unpinned Spans

In [795]:
%%sql

CREATE OR REPLACE FUNCTION redistribute_unpinned_spans(
    p_attribute_id INTEGER,
    p_modified_span_id INTEGER,
    p_new_weight DOUBLE PRECISION,
    p_old_weight DOUBLE PRECISION
) RETURNS VOID
LANGUAGE plpgsql
AS $$
DECLARE
    v_pinned_weight DOUBLE PRECISION;
    v_target         DOUBLE PRECISION;
BEGIN
    -- Compute the total weight of pinned discrete spans.
    SELECT COALESCE(SUM(weight), 0)
      INTO v_pinned_weight
      FROM span
     WHERE attribute_id = p_attribute_id
       AND type = 'discrete'
       AND is_percentage_pinned = true;
       
    -- The available weight for all unpinned spans.
    v_target := 1.0 - v_pinned_weight;
    
    -- If there’s only one unpinned span (v_target equals p_old_weight), then
    -- the new weight must equal the entire target.
    IF (v_target - p_old_weight) = 0 THEN
       IF p_new_weight <> v_target THEN
           RAISE EXCEPTION 'Cannot change weight; only one unpinned span exists and its weight must be %', v_target;
       END IF;
    END IF;
    
    -- Update all unpinned discrete spans:
    -- • The modified span gets the new weight.
    -- • All others are scaled proportionally.
    UPDATE span
       SET weight = CASE 
                      WHEN id = p_modified_span_id THEN p_new_weight
                      ELSE weight * ((v_target - p_new_weight) / (v_target - p_old_weight))
                    END
     WHERE attribute_id = p_attribute_id
       AND type = 'discrete'
       AND is_percentage_pinned = false;
END;
$$;

Running query in 'postgresql://notebook@localhost:5432/sahuagin'

++
||
++
++

## Add Discrete Span to Attribute

In [796]:
%%sql

CREATE OR REPLACE PROCEDURE add_disc_span_to_attr(
    p_attribute_id INTEGER,
    p_label VARCHAR(255)
)
LANGUAGE plpgsql
AS $$
DECLARE
    new_span_id     INTEGER;
    v_total_weight  DOUBLE PRECISION;
    v_pinned_weight DOUBLE PRECISION;
    v_target        DOUBLE PRECISION;
    v_count_old     INTEGER;
    candidate       DOUBLE PRECISION;
BEGIN
    -- Compute the total weight for all discrete spans for this attribute.
    SELECT COALESCE(SUM(weight), 0)
      INTO v_total_weight
      FROM span
     WHERE attribute_id = p_attribute_id
       AND type = 'discrete';

    -- If no spans exist, simply insert the first span with weight 1.
    IF v_total_weight = 0 THEN
        INSERT INTO span(attribute_id, label, type, is_percentage_pinned, weight)
        VALUES (p_attribute_id, p_label, 'discrete', false, 1.0);
        RETURN;
    END IF;
    
    -- Compute the total weight of pinned discrete spans.
    SELECT COALESCE(SUM(weight), 0)
      INTO v_pinned_weight
      FROM span
     WHERE attribute_id = p_attribute_id
       AND type = 'discrete'
       AND is_percentage_pinned = true;
    
    -- Calculate the available target weight for all unpinned spans.
    v_target := 1.0 - v_pinned_weight;
    IF v_target <= 0 THEN
        RAISE EXCEPTION 'No available weight for unpinned spans (v_target = %)', v_target;
    END IF;
    
    -- Count the existing non-pinned discrete spans.
    SELECT COUNT(*)
      INTO v_count_old
      FROM span
     WHERE attribute_id = p_attribute_id
       AND type = 'discrete'
       AND is_percentage_pinned = false;
    
    -- Insert the new unpinned span with a temporary weight of 0.
    INSERT INTO span(attribute_id, label, type, is_percentage_pinned, weight)
    VALUES (p_attribute_id, p_label, 'discrete', false, 0.0)
    RETURNING id INTO new_span_id;
    
    -- Compute the candidate weight for the new span.
    candidate := v_target / (v_count_old + 1);
    
    -- Redistribute the weights among all unpinned spans
    PERFORM redistribute_unpinned_spans(p_attribute_id, new_span_id, candidate, 0);
END;
$$;

Running query in 'postgresql://notebook@localhost:5432/sahuagin'

++
||
++
++

## Modify Discrete Span Weight

In [797]:
%%sql

CREATE OR REPLACE PROCEDURE modify_disc_span_weight(
    p_span_id    INTEGER,
    p_new_weight DOUBLE PRECISION
)
LANGUAGE plpgsql
AS $$
DECLARE
    v_attribute_id INTEGER;
    v_current_weight DOUBLE PRECISION;
    v_is_pinned    BOOLEAN;
    v_type         TEXT;
    v_pinned_weight DOUBLE PRECISION;
    v_target        DOUBLE PRECISION;
    v_other_sum     DOUBLE PRECISION;
BEGIN
    -- Get the current span's details.
    SELECT attribute_id, weight, is_percentage_pinned, type
      INTO v_attribute_id, v_current_weight, v_is_pinned, v_type
      FROM span
     WHERE id = p_span_id;
     
    -- Ensure the span exists, is of type 'discrete' and is not pinned.
    IF NOT FOUND OR v_type <> 'discrete' OR v_is_pinned THEN
       RAISE EXCEPTION 'Span % not found or not a modifiable discrete span', p_span_id;
    END IF;
    
    -- Compute the total pinned weight for this attribute.
    SELECT COALESCE(SUM(weight), 0)
      INTO v_pinned_weight
      FROM span
     WHERE attribute_id = v_attribute_id
       AND type = 'discrete'
       AND is_percentage_pinned = true;
       
    v_target := 1.0 - v_pinned_weight;
    
    -- Validate the new weight is between 0 and the available target.
    IF p_new_weight < 0 OR p_new_weight > v_target THEN
       RAISE EXCEPTION 'Invalid new weight: % (must be between 0 and %)', p_new_weight, v_target;
    END IF;
    
    -- Compute the total weight for all other unpinned spans.
    SELECT COALESCE(SUM(weight), 0)
      INTO v_other_sum
      FROM span
     WHERE attribute_id = v_attribute_id
       AND type = 'discrete'
       AND is_percentage_pinned = false
       AND id <> p_span_id;
       
    IF v_other_sum = 0 THEN
       -- There is only one unpinned span. Its weight must equal the available target.
       IF p_new_weight <> v_target THEN
         RAISE EXCEPTION 'Only one unpinned span exists. Its weight must be %', v_target;
       ELSE
         UPDATE span SET weight = p_new_weight WHERE id = p_span_id;
       END IF;
    ELSE
       -- Use the helper function to redistribute weights.
       PERFORM redistribute_unpinned_spans(v_attribute_id, p_span_id, p_new_weight, v_current_weight);
    END IF;
END;
$$;

Running query in 'postgresql://notebook@localhost:5432/sahuagin'

++
||
++
++

## Continuous Attribute

### Add

In [798]:
%%sql

CREATE OR REPLACE PROCEDURE add_continuous_attribute(
    p_name           VARCHAR(255),
    p_min_value      DOUBLE PRECISION,
    p_mode_value     DOUBLE PRECISION,
    p_max_value      DOUBLE PRECISION,
    p_concentration  DOUBLE PRECISION,
    p_skew           DOUBLE PRECISION,
    p_decimals       INTEGER,
    p_has_labels     BOOLEAN,
    p_has_value      BOOLEAN,
    p_units          VARCHAR(255),
    p_spans          JSON DEFAULT NULL  -- JSON array shortcut for continuous spans
)
LANGUAGE plpgsql
AS $$
DECLARE
    v_attr_id   INTEGER;
    span_count  INTEGER;
    i           INTEGER;
    current_min DOUBLE PRECISION;
    new_max     DOUBLE PRECISION;
    span_label  VARCHAR(255);
    spans_json  JSON;
BEGIN
    -- Insert the continuous attribute record.
    INSERT INTO attribute
      (name, type, decimals, has_labels, has_value, max_value, min_value,
       mode_value, concentration, skew, units)
    VALUES
      (p_name, 'continuous', p_decimals, p_has_labels, p_has_value, p_max_value,
       p_min_value, p_mode_value, p_concentration, p_skew, p_units)
    RETURNING id INTO v_attr_id;
    
    IF p_spans IS NOT NULL THEN
        spans_json := p_spans::json;
        span_count := json_array_length(spans_json);
        current_min := p_min_value;
        
        FOR i IN 0..span_count - 1 LOOP
            -- Each JSON tuple is assumed to be [label, new_max_value].
            span_label := (spans_json->i)->>0;
            new_max := ((spans_json->i)->>1)::DOUBLE PRECISION;
            
            -- Validate that the new maximum is in the valid interval.
            IF new_max <= current_min OR new_max > p_max_value THEN
                RAISE EXCEPTION 'Invalid span range: new_max (%) is not in the valid interval (% - %)',
                    new_max, current_min, p_max_value;
            END IF;
            
            -- Insert the continuous span.
            INSERT INTO span(attribute_id, label, type, min_value, max_value)
            VALUES(v_attr_id, span_label, 'continuous', current_min, new_max);
            
            current_min := new_max;
        END LOOP;
        
        -- If the JSON array does not reach the attribute's overall max, raise an exception.
        IF current_min < p_max_value THEN
            RAISE EXCEPTION 'Incomplete continuous span definitions: final span max is % which is less than the attribute maximum %', current_min, p_max_value;
        END IF;
    END IF;
END;
$$;

Running query in 'postgresql://notebook@localhost:5432/sahuagin'

++
||
++
++

### Get

In [799]:
%%sql

CREATE OR REPLACE FUNCTION get_continuous_attribute(p_attribute_name varchar)
RETURNS TABLE (
    name           varchar,
    type           attribute_type,
    min            double precision,
    mode           double precision,
    max            double precision,
    concentration  double precision,
    skew           double precision,
    units          varchar,
    decimals       integer,
    has_value      boolean,
    has_labels     boolean,
    span_label     varchar,
    span_min       double precision,
    span_max       double precision
)
LANGUAGE plpgsql
AS
$$
BEGIN
  RETURN QUERY
    SELECT 
      a.name,
      a.type,
      a.min_value,
      a.mode_value,
      a.max_value,
      a.concentration,
      a.skew,
      a.units,
      a.decimals,
      a.has_value,
      a.has_labels,
      s.label,
      s.min_value,
      s.max_value
    FROM attribute a
    LEFT JOIN span s ON s.attribute_id = a.id
    WHERE a.name = p_attribute_name
      AND a.type = 'continuous';
END;
$$;

Running query in 'postgresql://notebook@localhost:5432/sahuagin'

++
||
++
++

### Delete

In [800]:
%%sql

CREATE OR REPLACE PROCEDURE delete_continuous_attribute(p_attribute_name varchar)
LANGUAGE plpgsql
AS
$$
BEGIN
  DELETE FROM attribute
  WHERE name = p_attribute_name
    AND type = 'continuous';
END;
$$;

Running query in 'postgresql://notebook@localhost:5432/sahuagin'

++
||
++
++

## Add Continuous Span to Attribute

In [801]:
%%sql

CREATE OR REPLACE PROCEDURE add_cont_span_to_attr(
    p_attribute_id integer,
    p_label varchar(255),
    p_new_min double precision,
    p_new_max double precision
)
LANGUAGE plpgsql
AS $$
DECLARE
    v_attr_min double precision;
    v_attr_max double precision;
    v_count integer;
    v_first_id integer;
    v_first_min double precision;
    v_first_max double precision;
    v_last_id integer;
    v_last_min double precision;
    v_last_max double precision;
BEGIN
    -- Get attribute’s overall range (continuous only)
    SELECT min_value, max_value
      INTO v_attr_min, v_attr_max
      FROM attribute
     WHERE id = p_attribute_id
       AND type = 'continuous'
     LIMIT 1;
     
    IF v_attr_min IS NULL THEN
        RAISE EXCEPTION 'Attribute not found or not continuous';
    END IF;
    
    -- Validate requested span
    IF p_new_min < v_attr_min OR p_new_max > v_attr_max OR p_new_min >= p_new_max THEN
        RAISE EXCEPTION 'Invalid span range';
    END IF;
    
    -- If no continuous spans exist, insert one covering the full range
    SELECT COUNT(*) INTO v_count
      FROM span
     WHERE attribute_id = p_attribute_id
       AND type = 'continuous';
       
    IF v_count = 0 THEN
        INSERT INTO span(attribute_id, label, type, min_value, max_value)
        VALUES(p_attribute_id, p_label, 'continuous', v_attr_min, v_attr_max);
        RETURN;
    END IF;
    
    -- Adjust overlapping spans: first overlapping span (ordered by min_value)
    SELECT id, min_value, max_value
      INTO v_first_id, v_first_min, v_first_max
      FROM span
     WHERE attribute_id = p_attribute_id
       AND type = 'continuous'
       AND max_value > p_new_min
       AND min_value < p_new_max
     ORDER BY min_value ASC
     LIMIT 1;
     
    IF v_first_min < p_new_min THEN
        UPDATE span SET max_value = p_new_min WHERE id = v_first_id;
    END IF;
    
    -- Adjust overlapping spans: last overlapping span (ordered by max_value)
    SELECT id, min_value, max_value
      INTO v_last_id, v_last_min, v_last_max
      FROM span
     WHERE attribute_id = p_attribute_id
       AND type = 'continuous'
       AND max_value > p_new_min
       AND min_value < p_new_max
     ORDER BY max_value DESC
     LIMIT 1;
     
    IF v_last_max > p_new_max THEN
        UPDATE span SET min_value = p_new_max WHERE id = v_last_id;
    END IF;
    
    -- Remove spans fully covered by the new span
    DELETE FROM span
     WHERE attribute_id = p_attribute_id
       AND type = 'continuous'
       AND min_value >= p_new_min
       AND max_value <= p_new_max;
       
    INSERT INTO span(attribute_id, label, type, min_value, max_value)
    VALUES(p_attribute_id, p_label, 'continuous', p_new_min, p_new_max);
END;
$$;

Running query in 'postgresql://notebook@localhost:5432/sahuagin'

++
||
++
++

## Link Attribute to Variant

In [856]:
%%sql

CREATE OR REPLACE PROCEDURE link_attr_to_variant(
    p_variant_id   INTEGER,
    p_attribute_id INTEGER,
    p_name         VARCHAR,
    p_position     INTEGER DEFAULT NULL
)
LANGUAGE plpgsql
AS $$
DECLARE
    v_count    INTEGER;
    v_position INTEGER;
BEGIN
    SELECT COUNT(*) INTO v_count
      FROM variant_attribute
     WHERE variant_id = p_variant_id;
    
    IF p_position IS NULL THEN
        v_position := v_count;  -- append at end
    ELSE
        v_position := p_position;
        IF v_position < 0 THEN 
            v_position := 0;
        END IF;
        IF v_position > v_count THEN
            v_position := v_count;
        END IF;
        UPDATE variant_attribute
           SET causation_index = causation_index + 1
         WHERE variant_id = p_variant_id
           AND causation_index >= v_position;
    END IF;

    INSERT INTO variant_attribute(attribute_id, name, causation_index, variant_id)
    VALUES (p_attribute_id, p_name, v_position, p_variant_id);
END;
$$;

Running query in 'postgresql://notebook@localhost:5432/sahuagin'

++
||
++
++

## Variant

### Add

In [853]:
%%sql

CREATE OR REPLACE PROCEDURE add_variant(
    in_variant_name VARCHAR,
    in_attr_keys    JSON
)
LANGUAGE plpgsql
AS $$
DECLARE
    new_variant_id INTEGER;
    i              INTEGER := 0;
    key_count      INTEGER;
    attr_id        INTEGER;
    attr_name      VARCHAR;
BEGIN
    INSERT INTO variant (name)
    VALUES (in_variant_name)
    RETURNING id INTO new_variant_id;

    IF in_attr_keys IS NOT NULL THEN
        key_count := json_array_length(in_attr_keys::json);
        WHILE i < key_count LOOP
            -- Get the attribute name from the JSON array at position i.
            attr_name := in_attr_keys::json ->> i;

            -- Look up the attribute id for the given name.
            SELECT id
              INTO attr_id
              FROM attribute
             WHERE name = attr_name
             LIMIT 1;

            IF attr_id IS NULL THEN
                RAISE EXCEPTION 'Attribute with name "%" not found', attr_name;
            END IF;

            -- Call the helper procedure with the looked up attribute id.
            CALL link_attr_to_variant(new_variant_id, attr_id, attr_name);
            i := i + 1;
        END LOOP;
    END IF;
END;
$$;

Running query in 'postgresql://notebook@localhost:5432/sahuagin'

++
||
++
++

### Get

In [ ]:
%%sql

CREATE OR REPLACE FUNCTION get_variant_page(
    p_variant_name VARCHAR,
    p_page INTEGER,
    p_page_size INTEGER
)
RETURNS TABLE (
    overall_index     BIGINT,
    label             TEXT,
    activated_effects JSON,
    to_modify_effects JSON
)
LANGUAGE plpgsql
AS $$
BEGIN
  RETURN QUERY
  WITH RECURSIVE variant_id_cte AS (
      -- Look up the variant’s id by its unique name.
      SELECT id
      FROM variant
      WHERE name = p_variant_name
  ),
  recursive_flattened AS (
      -- Base step: get all top‑level attributes for the variant.
      SELECT 
          va.id AS variant_attribute_id,
          vas.id AS variant_attr_span_id,
          va.name AS attribute_name,
          va.causation_index,
          ARRAY[va.causation_index] AS flattened_path,
          (va.name || 
             CASE 
               WHEN s.label IS NOT NULL THEN '|' || s.label 
               ELSE '' 
             END
          ) AS agg_label,
          va.variant_id AS current_variant_id,
          -- Try to “resolve” this node by picking up an existing vavspan_attr record,
          -- using NULL for the parent since this is top‐level.
          rva.id AS resolved_id
      FROM variant_attribute va
      LEFT JOIN variant_attr_span vas 
          ON vas.variant_attribute_id = va.id
      LEFT JOIN span s 
          ON s.id = vas.span_id
      LEFT JOIN vavspan_attr rva 
          ON rva.variant_attribute_id = va.id
             AND rva.variant_attr_span_id = vas.id
             AND rva.parent_vavspan_attr_id IS NULL
      WHERE va.variant_id = (SELECT id FROM variant_id_cte)
      
      UNION ALL
      
      -- Recursive step: for each resolved node that activates a sub-variant, get its children.
      SELECT 
          va_child.id AS variant_attribute_id,
          vas_child.id AS variant_attr_span_id,
          va_child.name AS attribute_name,
          va_child.causation_index,
          rf.flattened_path || va_child.causation_index AS flattened_path,
          rf.agg_label || '/' || (va_child.name ||
             CASE 
               WHEN s_child.label IS NOT NULL THEN '|' || s_child.label 
               ELSE '' 
             END
          ) AS agg_label,
          va_child.variant_id AS current_variant_id,
          rva_child.id AS resolved_id
      FROM recursive_flattened rf
      -- Use the parent's chosen variant_attr_span (which “activates” a sub-variant)
      JOIN variant_attr_span vas_parent 
          ON vas_parent.variant_attribute_id = rf.variant_attribute_id
          AND vas_parent.id = rf.variant_attr_span_id
      -- A non-null vas_parent.variant_id indicates the activated sub-variant.
      JOIN variant_attribute va_child 
          ON va_child.variant_id = vas_parent.variant_id
      LEFT JOIN variant_attr_span vas_child 
          ON vas_child.variant_attribute_id = va_child.id
      LEFT JOIN span s_child 
          ON s_child.id = vas_child.span_id
      LEFT JOIN vavspan_attr rva_child 
          ON rva_child.variant_attribute_id = va_child.id
             AND rva_child.variant_attr_span_id = vas_child.id
             AND rva_child.parent_vavspan_attr_id = rf.resolved_id
      WHERE vas_parent.variant_id IS NOT NULL
  ),
  ordered AS (
      SELECT 
          row_number() OVER (ORDER BY flattened_path) AS overall_index,
          agg_label AS label,
          resolved_id,
          flattened_path
      FROM recursive_flattened
  ),
  paged AS (
      SELECT *
      FROM ordered
      ORDER BY flattened_path
      LIMIT p_page_size OFFSET (p_page - 1) * p_page_size
  )
  SELECT 
      p.overall_index,
      p.label,
      (
         SELECT COALESCE(json_agg(e.name ORDER BY e.name), '[]'::json)
         FROM variation v
         JOIN effect e ON e.variation_id = v.id
         WHERE v.activating_vavspan_attr_id = p.resolved_id
      ) AS activated_effects,
      (
         SELECT COALESCE(json_agg(e.name ORDER BY e.name), '[]'::json)
         FROM variation v
         JOIN effect e ON e.variation_id = v.id
         WHERE v.to_modify_vavspan_attr_id = p.resolved_id
      ) AS to_modify_effects
  FROM paged p
  ORDER BY p.overall_index;
END;
$$;

Running query in 'postgresql://notebook@localhost:5432/sahuagin'

++
||
++
++

### Delete

In [805]:
%%sql

CREATE OR REPLACE PROCEDURE delete_variant(p_variant_name varchar)
LANGUAGE plpgsql
AS
$$
BEGIN
  DELETE FROM variant WHERE name = p_variant_name;
END;
$$;

Running query in 'postgresql://notebook@localhost:5432/sahuagin'

++
||
++
++

## Add Entity

In [806]:
%%sql

CREATE OR REPLACE PROCEDURE add_entity(
    in_name VARCHAR(255),
    in_variant_id INT
)
LANGUAGE plpgsql
AS $$
BEGIN
    INSERT INTO entity(name, variant_id)
    VALUES (in_name, in_variant_id);
END;
$$;

Running query in 'postgresql://notebook@localhost:5432/sahuagin'

++
||
++
++

## Roll Discrete Variant Attribute Value

In [807]:
%%sql

CREATE OR REPLACE FUNCTION roll_discrete_varattr(
    p_vavspan_attr_id INTEGER,
    p_exclude_span_id INTEGER
) 
RETURNS INTEGER
LANGUAGE plpgsql
AS $$
DECLARE
    v_span_id INTEGER;
    v_max_double constant double precision := 1e308;
    v_variant_attribute_id INTEGER;
    v_variant_attr_span_id INTEGER;
BEGIN
    -- Retrieve the underlying variant attribute and its associated variant_attr_span.
    SELECT variant_attribute_id, variant_attr_span_id
      INTO v_variant_attribute_id, v_variant_attr_span_id
      FROM vavspan_attr
     WHERE id = p_vavspan_attr_id;
    
    WITH
    BaseSpans AS (
         SELECT s.id AS span_id, 
                va.id AS variant_attribute_id, 
                vas.id AS variant_attr_span_id, 
                s.weight AS base_weight
         FROM variant_attribute va
         JOIN span s ON s.attribute_id = va.attribute_id
         LEFT JOIN variant_attr_span vas 
           ON vas.variant_attribute_id = va.id 
          AND vas.span_id = s.id
         WHERE va.id = v_variant_attribute_id
           AND s.type = 'discrete'
           AND (p_exclude_span_id = 0 OR s.id <> p_exclude_span_id)
    ), 
    ActiveVariations AS (
         -- Only consider variations that have an effect.
         SELECT v.id AS variation_id, 
                vav.variant_attribute_id,
                eff.id AS effect_id
         FROM variation v
         JOIN effect eff 
           ON eff.variation_id = v.id
         JOIN vavspan_attr vav 
           ON vav.id = v.to_modify_vavspan_attr_id
          AND vav.variant_attribute_id = v_variant_attribute_id
         WHERE vav.id = p_vavspan_attr_id
    ), 
    InactiveSpans AS (
         SELECT DISTINCT vis.span_id
         FROM variation_inactive_span vis
         JOIN ActiveVariations av 
           ON av.effect_id = vis.effect_id
    ), 
    ActivatedSpans AS (
         SELECT DISTINCT 
                vas.span_id, 
                av.variant_attribute_id, 
                NULL::INTEGER AS variant_attr_span_id, 
                0.0 AS base_weight
         FROM variation_activated_span vas
         JOIN ActiveVariations av 
           ON av.effect_id = vas.effect_id
    ), 
    DeltaWeights AS (
         SELECT vdw.span_id, 
                SUM(vdw.delta_weight) AS total_delta
         FROM variation_delta_weight vdw
         JOIN ActiveVariations av 
           ON av.effect_id = vdw.effect_id
         GROUP BY vdw.span_id
    ), 
    AllRelevantSpans AS (
         SELECT b.span_id, b.variant_attribute_id, b.variant_attr_span_id, b.base_weight
         FROM BaseSpans b
         WHERE b.span_id NOT IN (SELECT span_id FROM InactiveSpans)
         UNION
         SELECT a.span_id, a.variant_attribute_id, a.variant_attr_span_id, a.base_weight
         FROM ActivatedSpans a
         WHERE a.span_id NOT IN (SELECT span_id FROM InactiveSpans)
    ), 
    FinalSpans AS (
         SELECT ars.span_id, 
                ars.variant_attribute_id, 
                ars.variant_attr_span_id,
                COALESCE(ars.base_weight, 0) + COALESCE(dw.total_delta, 0) AS effective_weight
         FROM AllRelevantSpans ars
         LEFT JOIN DeltaWeights dw 
           ON dw.span_id = ars.span_id
    ),
    SpanCounts AS (
         SELECT ns.num_spans, 
                SUM(fs.effective_weight / ns.num_spans) AS total_weight
         FROM FinalSpans fs
         CROSS JOIN (
             SELECT COUNT(*)::double precision AS num_spans 
             FROM FinalSpans 
             WHERE effective_weight > 0
         ) ns
         WHERE fs.effective_weight > 0
         GROUP BY ns.num_spans
    ),
    AdjustedSpans AS (
         SELECT fs.span_id,
                fs.variant_attribute_id,
                fs.variant_attr_span_id,
                fs.effective_weight,
                sc.num_spans,
                sc.total_weight,
                CASE 
                  WHEN fs.effective_weight < 1 
                    THEN ceil((fs.effective_weight * v_max_double) / (sc.total_weight * sc.num_spans))
                  ELSE ceil((fs.effective_weight / (sc.total_weight * sc.num_spans)) * v_max_double)
                END AS adjusted_weight
         FROM FinalSpans fs
         CROSS JOIN SpanCounts sc
         WHERE fs.effective_weight > 0
    ),
    Running AS (
         SELECT span_id,
                variant_attribute_id,
                variant_attr_span_id,
                adjusted_weight,
                SUM(adjusted_weight) OVER (ORDER BY span_id) AS running_total
         FROM AdjustedSpans
    ),
    OrderedSpansCTE AS (
         SELECT span_id,
                variant_attribute_id,
                variant_attr_span_id,
                adjusted_weight AS contextual_weight,
                running_total,
                LAG(running_total, 1, 0) OVER (ORDER BY span_id) AS prev_running_total
         FROM Running
    ),
    TotalAdjusted AS (
         SELECT MAX(running_total) AS v_total_weight
         FROM OrderedSpansCTE
    ),
    Rng AS (
         SELECT t.v_total_weight,
                floor(random() * t.v_total_weight) AS v_random_pick
         FROM TotalAdjusted t
    )
    SELECT o.span_id
      INTO v_span_id
      FROM OrderedSpansCTE o, Rng
      WHERE Rng.v_random_pick >= o.prev_running_total
        AND Rng.v_random_pick < o.running_total
      LIMIT 1;

    RETURN v_span_id;
END;
$$;

Running query in 'postgresql://notebook@localhost:5432/sahuagin'

++
||
++
++

## Gamma Range

In [808]:
%%sql

CREATE OR REPLACE FUNCTION gamma_rng(shape double precision)
RETURNS double precision
LANGUAGE plpgsql
AS $$
DECLARE
    d double precision;
    c double precision;
    x double precision;
    v double precision;
    u double precision;
BEGIN
    IF shape < 1 THEN
        -- Use the transformation: Gamma(shape) = Gamma(shape+1)*U^(1/shape)
        RETURN gamma_rng(shape + 1) * power(random(), 1.0 / shape);
    END IF;
    d := shape - 1.0/3.0;
    c := 1.0 / sqrt(9.0 * d);
    LOOP
        -- Generate a standard normal via Box–Muller:
        x := sqrt(-2 * ln(random())) * cos(2 * 3.141592653589793 * random());
        v := 1 + c * x;
        IF v <= 0 THEN
            CONTINUE;
        END IF;
        v := v * v * v;
        u := random();
        IF u < 1 - 0.0331 * x * x * x * x THEN
            RETURN d * v;
        END IF;
        IF ln(u) < 0.5 * x * x + d * (1 - v + ln(v)) THEN
            RETURN d * v;
        END IF;
    END LOOP;
END;
$$;

Running query in 'postgresql://notebook@localhost:5432/sahuagin'

++
||
++
++

## Generate Weighted Random Number

In [809]:
%%sql

-- using a fast Marsaglia–Tsang method
CREATE OR REPLACE FUNCTION weighted_random(
    min_val double precision,                   -- Minimum of desired range.
    max_val double precision,                   -- Maximum of desired range (must be > min_val).
    mode_val double precision,                  -- Desired mode (peak) in the same units.
    skew double precision DEFAULT 0,            -- Skew adjustment; between -1 and 1 only, 0 = no skew.
    concentration double precision DEFAULT 4    -- Concentration (k); should be >2 for a proper unimodal shape.
) RETURNS double precision
LANGUAGE plpgsql
AS $$
DECLARE
    m_norm      double precision;  -- Normalized desired mode in [0,1].
    alpha       double precision;
    beta        double precision;
    x_val       double precision;
    y_val       double precision;
    beta_sample double precision;
BEGIN
    IF min_val >= max_val THEN
        RAISE EXCEPTION 'min_val must be less than max_val';
    END IF;
    IF concentration < 0 THEN
        RAISE EXCEPTION 'concentration must be non-negative';
    END IF;
    IF concentration <= 2 THEN
        -- When concentration <= 2 the Beta isn't unimodal; fallback to uniform.
        RETURN min_val + random() * (max_val - min_val);
    END IF;

    -- Normalize the desired mode to [0,1].
    m_norm := (mode_val - min_val) / (max_val - min_val);

    -- Parameterize α and β so that (α-1)/(α+β-2) equals m_norm.
    alpha := m_norm * (concentration - 2) + 1;
    beta  := (1 - m_norm) * (concentration - 2) + 1;

    -- Apply skew correction if desired.
    IF skew <> 0 THEN
        alpha := alpha * (1 + skew);
        beta  := beta  * (1 - skew);
    END IF;

    -- Generate a Beta(α,β) random variable via the Gamma method.
    x_val := gamma_rng(alpha);
    y_val := gamma_rng(beta);
    beta_sample := x_val / (x_val + y_val);

    -- Scale the [0,1] Beta sample to [min_val, max_val].
    RETURN min_val + beta_sample * (max_val - min_val);
END;
$$;

Running query in 'postgresql://notebook@localhost:5432/sahuagin'

++
||
++
++

## Roll Continuous Variant Attribute Value

In [810]:
%%sql

CREATE OR REPLACE FUNCTION roll_continuous_varattr(
    p_vavspan_attr_id INTEGER,
    p_exclude_span_id  INTEGER
)
RETURNS TABLE(chosen_span_id integer, chosen_value double precision)
LANGUAGE plpgsql
AS $$
DECLARE
    v_attribute_id         integer;
    v_decimals             integer;
    v_min                  double precision;
    v_max                  double precision;
    v_mode                 double precision;
    v_concentration        double precision;
    v_skew                 double precision;
    v_total_delta_mode     double precision := 0;
    v_total_delta_conc     double precision := 0;
    v_total_delta_skew     double precision := 0;
    v_eff_min              double precision;
    v_eff_max              double precision;
    v_eff_mode             double precision;
    v_result               double precision;
    v_clamped_result       double precision;
    v_chosen_span_id       integer;
    dummy_eff_min          double precision;
    dummy_eff_max          double precision;
    v_variant_attribute_id integer;
BEGIN
    -- Retrieve attribute details from the vavspan_attr record.
    SELECT va.attribute_id, a.decimals, a.min_value, a.max_value, a.mode_value, a.concentration, a.skew, va.id
      INTO v_attribute_id, v_decimals, v_min, v_max, v_mode, v_concentration, v_skew, v_variant_attribute_id
      FROM vavspan_attr vav
      JOIN variant_attribute va ON va.id = vav.variant_attribute_id
      JOIN attribute a ON a.id = va.attribute_id
     WHERE vav.id = p_vavspan_attr_id
     LIMIT 1;

    IF v_attribute_id IS NULL THEN
        RAISE EXCEPTION 'No matching attribute found';
    END IF;

    -- Sum variation deltas by joining through the effect table.
    SELECT COALESCE(SUM(vca.delta_mode), 0),
           COALESCE(SUM(vca.delta_conc), 0),
           COALESCE(SUM(vca.delta_skew), 0)
      INTO v_total_delta_mode, v_total_delta_conc, v_total_delta_skew
      FROM variation_continuous_attr vca
      JOIN effect e ON e.id = vca.effect_id
      JOIN variation v ON v.id = e.variation_id
      JOIN vavspan_attr vav ON vav.id = v.to_modify_vavspan_attr_id
     WHERE vav.variant_attribute_id = v_variant_attribute_id
       AND vav.id = p_vavspan_attr_id;

    -- Compute effective min, max, and mode.
    IF (v_total_delta_conc <> 0 OR v_total_delta_skew <> 0) THEN
        v_eff_min  := (v_min + v_total_delta_mode) * (1 + v_total_delta_conc + v_total_delta_skew);
        v_eff_max  := (v_max + v_total_delta_mode) * (1 + v_total_delta_conc + v_total_delta_skew);
        v_eff_mode := (v_mode + v_total_delta_mode) * (1 + v_total_delta_conc + v_total_delta_skew);
    ELSE
        v_eff_min  := v_min + v_total_delta_mode;
        v_eff_max  := v_max + v_total_delta_mode;
        v_eff_mode := v_mode + v_total_delta_mode;
    END IF;

    IF v_eff_max < v_eff_min THEN
        -- Swap if needed.
        v_result  := v_eff_min;
        v_eff_min := v_eff_max;
        v_eff_max := v_result;
    END IF;

    -- Generate a weighted random value.
    v_result := weighted_random(
                   v_eff_min, 
                   v_eff_max, 
                   v_eff_mode, 
                   COALESCE(v_skew, 0),
                   COALESCE(v_concentration, 0)
               );
    v_result := round(v_result::numeric, v_decimals)::double precision;

    -- Clamp the result to the effective range.
    IF v_result < v_eff_min THEN
        v_clamped_result := v_eff_min;
    ELSIF v_result > v_eff_max THEN
        v_clamped_result := v_eff_max;
    ELSE
        v_clamped_result := v_result;
    END IF;

    -- Select the span matching the computed result.
    IF p_exclude_span_id IS NOT NULL AND p_exclude_span_id <> 0 THEN
        SELECT s.id,
               CASE WHEN (v_total_delta_conc <> 0 OR v_total_delta_skew <> 0)
                    THEN (s.min_value + v_total_delta_mode) * (1 + v_total_delta_conc + v_total_delta_skew)
                    ELSE (s.min_value + v_total_delta_mode)
               END,
               CASE WHEN (v_total_delta_conc <> 0 OR v_total_delta_skew <> 0)
                    THEN (s.max_value + v_total_delta_mode) * (1 + v_total_delta_conc + v_total_delta_skew)
                    ELSE (s.max_value + v_total_delta_mode)
               END
          INTO v_chosen_span_id, dummy_eff_min, dummy_eff_max
          FROM span s
          JOIN variant_attr_span vas ON vas.span_id = s.id
         WHERE s.attribute_id = v_attribute_id
           AND s.type = 'continuous'
           AND s.id <> p_exclude_span_id
           AND vas.id = p_vavspan_attr_id
           AND (CASE WHEN (v_total_delta_conc <> 0 OR v_total_delta_skew <> 0)
                     THEN (s.min_value + v_total_delta_mode) * (1 + v_total_delta_conc + v_total_delta_skew)
                     ELSE (s.min_value + v_total_delta_mode)
                END) <= v_clamped_result
           AND (CASE WHEN (v_total_delta_conc <> 0 OR v_total_delta_skew <> 0)
                     THEN (s.max_value + v_total_delta_mode) * (1 + v_total_delta_conc + v_total_delta_skew)
                     ELSE (s.max_value + v_total_delta_mode)
                END) > v_clamped_result
         LIMIT 1;
    ELSE
        SELECT s.id,
               CASE WHEN (v_total_delta_conc <> 0 OR v_total_delta_skew <> 0)
                    THEN (s.min_value + v_total_delta_mode) * (1 + v_total_delta_conc + v_total_delta_skew)
                    ELSE (s.min_value + v_total_delta_mode)
               END,
               CASE WHEN (v_total_delta_conc <> 0 OR v_total_delta_skew <> 0)
                    THEN (s.max_value + v_total_delta_mode) * (1 + v_total_delta_conc + v_total_delta_skew)
                    ELSE (s.max_value + v_total_delta_mode)
               END
          INTO v_chosen_span_id, dummy_eff_min, dummy_eff_max
          FROM span s
          JOIN variant_attr_span vas ON vas.span_id = s.id
         WHERE s.attribute_id = v_attribute_id
           AND s.type = 'continuous'
           AND vas.id = p_vavspan_attr_id
           AND (CASE WHEN (v_total_delta_conc <> 0 OR v_total_delta_skew <> 0)
                     THEN (s.min_value + v_total_delta_mode) * (1 + v_total_delta_conc + v_total_delta_skew)
                     ELSE (s.min_value + v_total_delta_mode)
                END) <= v_clamped_result
           AND (CASE WHEN (v_total_delta_conc <> 0 OR v_total_delta_skew <> 0)
                     THEN (s.max_value + v_total_delta_mode) * (1 + v_total_delta_conc + v_total_delta_skew)
                     ELSE (s.max_value + v_total_delta_mode)
                END) > v_clamped_result
         LIMIT 1;
    END IF;

    chosen_span_id := v_chosen_span_id;
    chosen_value   := v_clamped_result;
    RETURN NEXT;
    RETURN;
END;
$$;

Running query in 'postgresql://notebook@localhost:5432/sahuagin'

++
||
++
++

## Generate Entity State

In [811]:
%%sql

CREATE OR REPLACE PROCEDURE generate_entity_state(
    IN  p_entity_id                  INTEGER,
    IN  p_time                       DOUBLE PRECISION,
    IN  p_regenerate_entity_state_id INTEGER
)
LANGUAGE plpgsql
AS $$
DECLARE
    v_root_variant_id         INTEGER;
    v_entity_state_id         INTEGER;
    v_current_variant         INTEGER;
    v_trigger_vas             INTEGER;  -- the variant_attr_span id that triggered this variant (NULL for top-level)
    v_parent_vavspan_attr_id  INTEGER;  -- the parent's vavspan_attr id (NULL for top-level)
    cur_va_id               INTEGER;
    cur_attr_type           TEXT;
    v_variant_attr_span_id  INTEGER;
    v_existing_evav_id      INTEGER;
    v_existing_span_id      INTEGER;
    v_lock_count            INTEGER := 0;
    v_new_span_id           INTEGER;
    v_new_numeric           DOUBLE PRECISION;
    v_used_span_id          INTEGER;
    v_sub_variant_id        INTEGER;
    v_attr_counter          INTEGER := 0;
    v_queue_id              INTEGER;  -- for stack (LIFO) ordering
    cur_vavspan_attr_id     INTEGER;  -- resolved unique vavspan_attr id for this combination
    rec                     RECORD;
BEGIN
    -- Get the root variant from the entity.
    SELECT variant_id
      INTO v_root_variant_id
      FROM entity
     WHERE id = p_entity_id;
    IF v_root_variant_id IS NULL THEN
       RAISE EXCEPTION 'No entity found with id=%', p_entity_id;
    END IF;

    -- Use supplied state id (for regeneration) or insert a new entity_state.
    IF p_regenerate_entity_state_id IS NOT NULL THEN
        v_entity_state_id := p_regenerate_entity_state_id;
    ELSE
        INSERT INTO entity_state(entity_id, "time")
        VALUES (p_entity_id, p_time)
        RETURNING id INTO v_entity_state_id;
    END IF;

    ------------------------------
    -- Create a temporary queue that holds variants to process.
    -- Queue includes:
    --   variant_id: the variant whose attributes will be generated.
    --   variant_attr_span_id: the span that triggered this variant (may be NULL).
    --   parent_vavspan_attr_id: the parent's vavspan_attr id (NULL for top-level).
    ------------------------------
    DROP TABLE IF EXISTS _variant_queue;
    CREATE TEMPORARY TABLE _variant_queue (
        queue_id serial PRIMARY KEY,
        variant_id INTEGER,
        variant_attr_span_id INTEGER,
        parent_vavspan_attr_id INTEGER
    ) ON COMMIT DROP;
    -- Top-level variant: no triggering span and no parent.
    INSERT INTO _variant_queue (variant_id, variant_attr_span_id, parent_vavspan_attr_id)
      VALUES (v_root_variant_id, NULL, NULL);

    -- Temporary table to hold the current variant's attributes.
    DROP TABLE IF EXISTS _variant_attributes;
    CREATE TEMPORARY TABLE _variant_attributes (
        va_id    INTEGER,
        attr_type TEXT
    ) ON COMMIT DROP;

    -- Process the variant queue (LIFO for depth-first traversal).
    LOOP
        SELECT queue_id, variant_id, variant_attr_span_id, parent_vavspan_attr_id
          INTO v_queue_id, v_current_variant, v_trigger_vas, v_parent_vavspan_attr_id
          FROM _variant_queue
         ORDER BY queue_id DESC
         LIMIT 1;
        EXIT WHEN NOT FOUND;
        DELETE FROM _variant_queue WHERE queue_id = v_queue_id;

        -- Load the attributes for the current variant.
        TRUNCATE _variant_attributes;
        INSERT INTO _variant_attributes (va_id, attr_type)
          SELECT va.id, a.type
            FROM variant_attribute va
            JOIN attribute a ON a.id = va.attribute_id
           WHERE va.variant_id = v_current_variant
           ORDER BY va.causation_index ASC;

        v_attr_counter := 0;
        FOR rec IN
            SELECT va_id, attr_type FROM _variant_attributes
        LOOP
            cur_va_id     := rec.va_id;
            cur_attr_type := rec.attr_type;
            v_attr_counter := v_attr_counter + 1;

            -- Determine the variant_attr_span ID for this attribute.
            IF v_trigger_vas IS NOT NULL THEN
                SELECT id
                  INTO v_variant_attr_span_id
                  FROM variant_attr_span
                 WHERE id = v_trigger_vas
                   AND variant_attribute_id = cur_va_id
                 LIMIT 1;
            ELSE
                v_variant_attr_span_id := NULL;
            END IF;

            -- Resolve (or create) the unique vavspan_attr record for this combination.
            SELECT id
              INTO cur_vavspan_attr_id
              FROM vavspan_attr
             WHERE variant_attribute_id = cur_va_id
               AND ((parent_vavspan_attr_id IS NULL AND v_parent_vavspan_attr_id IS NULL)
                    OR parent_vavspan_attr_id = v_parent_vavspan_attr_id)
               AND ((variant_attr_span_id IS NULL AND v_variant_attr_span_id IS NULL)
                    OR variant_attr_span_id = v_variant_attr_span_id)
             LIMIT 1;
            IF NOT FOUND THEN
                INSERT INTO vavspan_attr (variant_attribute_id, variant_attr_span_id, parent_vavspan_attr_id)
                VALUES (cur_va_id, v_variant_attr_span_id, v_parent_vavspan_attr_id)
                RETURNING id INTO cur_vavspan_attr_id;
            END IF;

            -- Process or update the entity_varattr_value for this resolved vavspan_attr.
            IF p_regenerate_entity_state_id IS NOT NULL THEN
                SELECT id, span_id
                  INTO v_existing_evav_id, v_existing_span_id
                  FROM entity_varattr_value
                 WHERE entity_state_id = v_entity_state_id
                   AND vavspan_attr_id = cur_vavspan_attr_id
                 LIMIT 1;
            ELSE
                v_existing_evav_id := NULL;
                v_existing_span_id := NULL;
            END IF;

            IF p_regenerate_entity_state_id IS NOT NULL AND v_existing_evav_id IS NOT NULL THEN
                -- If a lock exists, re-use the existing span; otherwise roll a new one.
                IF EXISTS (
                     SELECT 1
                       FROM evav_lock
                      WHERE locked_evav_id = v_existing_evav_id
                ) THEN
                    v_used_span_id := v_existing_span_id;
                ELSE
                    IF cur_attr_type = 'discrete' THEN
                        v_new_span_id := roll_discrete_varattr(cur_vavspan_attr_id, 0);
                        UPDATE entity_varattr_value
                           SET span_id = v_new_span_id
                         WHERE id = v_existing_evav_id;
                        v_used_span_id := v_new_span_id;
                    ELSE
                        SELECT t.chosen_span_id, t.chosen_value
                          INTO v_new_span_id, v_new_numeric
                          FROM roll_continuous_varattr(cur_vavspan_attr_id, 0) t
                         LIMIT 1;
                        UPDATE entity_varattr_value
                           SET span_id = v_new_span_id,
                               numeric_value = v_new_numeric
                         WHERE id = v_existing_evav_id;
                        v_used_span_id := v_new_span_id;
                    END IF;
                END IF;
            ELSE
                IF cur_attr_type = 'discrete' THEN
                    v_new_span_id := roll_discrete_varattr(cur_vavspan_attr_id, 0);
                    INSERT INTO entity_varattr_value(
                        entity_state_id,
                        numeric_value,
                        span_id,
                        vavspan_attr_id
                    ) VALUES (
                        v_entity_state_id,
                        NULL,
                        v_new_span_id,
                        cur_vavspan_attr_id
                    );
                    v_used_span_id := v_new_span_id;
                ELSE
                    SELECT t.chosen_span_id, t.chosen_value
                      INTO v_new_span_id, v_new_numeric
                      FROM roll_continuous_varattr(cur_vavspan_attr_id, 0) t
                     LIMIT 1;
                    INSERT INTO entity_varattr_value(
                        entity_state_id,
                        numeric_value,
                        span_id,
                        vavspan_attr_id
                    ) VALUES (
                        v_entity_state_id,
                        v_new_numeric,
                        v_new_span_id,
                        cur_vavspan_attr_id
                    );
                    v_used_span_id := v_new_span_id;
                END IF;
            END IF;

            -- If the chosen span activates a sub–variant, enqueue that variant.
            SELECT variant_id
              INTO v_sub_variant_id
              FROM variant_attr_span
             WHERE variant_attribute_id = cur_va_id
               AND id = v_used_span_id
             LIMIT 1;
            IF v_sub_variant_id IS NOT NULL AND v_sub_variant_id <> v_current_variant THEN
                INSERT INTO _variant_queue (variant_id, variant_attr_span_id, parent_vavspan_attr_id)
                VALUES (v_sub_variant_id, v_used_span_id, cur_vavspan_attr_id)
                ON CONFLICT DO NOTHING;
            END IF;
        END LOOP;
    END LOOP;

    DROP TABLE IF EXISTS _variant_attributes;
    DROP TABLE IF EXISTS _variant_queue;
END;
$$;

Running query in 'postgresql://notebook@localhost:5432/sahuagin'

++
||
++
++

## Get Entity State Details

In [812]:
%%sql

DROP TYPE IF EXISTS entity_state_details CASCADE;
CREATE TYPE entity_state_details AS (
  "Index"                   BIGINT,
  "Attribute"               VARCHAR(255),
  "Value"                   DOUBLE PRECISION,
  "Units"                   VARCHAR(255),
  "Label"                   VARCHAR(255),
  "Active Variations"       JSONB,
  "Caused Variant"          VARCHAR(255),
  "Locked"                  BOOLEAN
);

CREATE OR REPLACE FUNCTION get_entity_state_details(
    p_entity_id INT,
    p_time      DOUBLE PRECISION
)
RETURNS SETOF entity_state_details
LANGUAGE plpgsql
AS $$
BEGIN
  RETURN QUERY
  WITH RECURSIVE 
  -- Compute a per–variant sibling order using causation_index.
  ordered_va AS (
    SELECT
      va.id AS variant_attribute_id,
      va.variant_id,
      va.attribute_id,
      va.name,
      va.causation_index,
      ROW_NUMBER() OVER (PARTITION BY va.variant_id ORDER BY va.causation_index) AS sibling_order
    FROM variant_attribute va
  ),
  --
  -- Build the processing tree while computing two “paths”:
  --
  --   * attribute_path – the flattened path using "/" as separator and not including the span label.
  --     For example: Human/Male/Irish/Hair Color
  --
  --   * json_full_address – same as attribute_path, except that if a span label exists it is appended
  --     with a colon (e.g. Human/Male/Irish/Hair Color:Red). This is used as the key in Active Variations.
  --
  ev_tree AS (
    -- Anchor: start with the entity state’s root variant.
    SELECT
      es."time" AS state_time,
      ev.id AS ev_id,
      ev.entity_state_id,
      ov.variant_attribute_id,
      v.name || '/' || ov.name AS attribute_path,
      v.name || '/' || ov.name || COALESCE(':' || s.label, '') AS json_full_address,
      v.id AS variant_id,
      v.name AS variant_name,
      ev.numeric_value,
      a.units,
      ev.span_id,
      s.label AS span_label,
      vas.variant_id AS activated_sub_variant,
      ARRAY[ ov.sibling_order ] AS proc_path
    FROM entity_state es
    JOIN entity e ON es.entity_id = e.id
    JOIN variant v ON e.variant_id = v.id
    JOIN entity_varattr_value ev ON ev.entity_state_id = es.id
    -- Unwrap the vavspan_attr record to get the underlying variant_attribute
    JOIN vavspan_attr vav ON ev.vavspan_attr_id = vav.id
    JOIN ordered_va ov ON ov.variant_attribute_id = vav.variant_attribute_id
    JOIN attribute a ON a.id = ov.attribute_id
    LEFT JOIN span s ON ev.span_id = s.id
    LEFT JOIN variant_attr_span vas 
           ON vas.id = ev.span_id
          AND vas.variant_attribute_id = vav.variant_attribute_id
    WHERE e.id = p_entity_id
      AND es."time" = p_time
      AND ov.variant_id = v.id

    UNION ALL

    -- Recursive step: for any attribute that activated a sub–variant,
    -- process the sub–variant’s attributes.
    SELECT
      parent.state_time,
      ev.id AS ev_id,
      ev.entity_state_id,
      ov.variant_attribute_id,
      parent.attribute_path || '/' || ov.name AS attribute_path,
      parent.json_full_address || '/' || ov.name || COALESCE(':' || s.label, '') AS json_full_address,
      v_child.id AS variant_id,
      v_child.name AS variant_name,
      ev.numeric_value,
      a.units,
      ev.span_id,
      s.label AS span_label,
      vas.variant_id AS activated_sub_variant,
      parent.proc_path || ov.sibling_order AS proc_path
    FROM ev_tree parent
    JOIN entity_varattr_value ev ON ev.entity_state_id = parent.entity_state_id
    -- Unwrap vavspan_attr for this level as well.
    JOIN vavspan_attr vav ON ev.vavspan_attr_id = vav.id
    JOIN ordered_va ov ON ov.variant_attribute_id = vav.variant_attribute_id
    JOIN variant v_child ON ov.variant_id = v_child.id
    JOIN attribute a ON a.id = ov.attribute_id
    LEFT JOIN span s ON ev.span_id = s.id
    LEFT JOIN variant_attr_span vas 
           ON vas.id = ev.span_id
          AND vas.variant_attribute_id = vav.variant_attribute_id
    WHERE parent.activated_sub_variant IS NOT NULL
      AND ov.variant_id = parent.activated_sub_variant
  ),
  --
  -- Index the tree by ordering lexicographically on proc_path.
  ev_indexed AS (
    SELECT ev.*,
           row_number() OVER (ORDER BY ev.proc_path) AS effective_causation_index
    FROM ev_tree ev
  )
  --
  -- Final select: return one row per processed attribute value.
  SELECT 
    ev.effective_causation_index::bigint AS "Index",
    ev.attribute_path::varchar(255) AS "Attribute",
    ev.numeric_value::double precision AS "Value",
    ev.units::varchar(255) AS "Units",
    ev.span_label::varchar(255) AS "Label",
    (
      SELECT COALESCE(
        jsonb_object_agg(anc_variations.json_full_address, anc_variations.variations_array),
        '{}'::jsonb
      )
      FROM (
        SELECT
          anc.json_full_address,
          jsonb_agg(
            jsonb_build_object(
              'variation_id', v.id,
              'is_inactive', v.is_inactive,
              'variation_continuous_attr', CASE WHEN vca.id IS NOT NULL THEN to_jsonb(vca) ELSE NULL END,
              'variation_activated_span', CASE WHEN vas2.id IS NOT NULL THEN to_jsonb(vas2) ELSE NULL END,
              'variation_delta_weight', CASE WHEN vd.id IS NOT NULL THEN to_jsonb(vd) ELSE NULL END,
              'variation_inactive_span', CASE WHEN vi.id IS NOT NULL THEN to_jsonb(vi) ELSE NULL END
            )
          ) AS variations_array
        FROM ev_indexed anc
        JOIN variation v ON v.activating_span_id = anc.span_id
        LEFT JOIN variation_continuous_attr vca ON vca.variation_id = v.id
        LEFT JOIN variation_activated_span vas2 ON vas2.variation_id = v.id
        LEFT JOIN variation_delta_weight vd ON vd.variation_id = v.id
        LEFT JOIN variation_inactive_span vi ON vi.variation_id = v.id
        WHERE anc.effective_causation_index < ev.effective_causation_index
          AND anc.span_id IS NOT NULL
        GROUP BY anc.json_full_address
      ) anc_variations
    ) AS "Active Variations",
    (
      CASE 
        WHEN ev.activated_sub_variant IS NOT NULL THEN
           (SELECT v2.name FROM variant v2 WHERE v2.id = ev.activated_sub_variant)
        ELSE NULL
      END
    )::varchar(255) AS "Caused Variant",
    EXISTS (
      SELECT 1 
      FROM evav_lock 
      WHERE locked_evav_id = ev.ev_id
    ) AS "Locked"
  FROM ev_indexed ev
  ORDER BY ev.effective_causation_index;
END;
$$;

Running query in 'postgresql://notebook@localhost:5432/sahuagin'

++
||
++
++

## Generate Entity Group

In [813]:
%%sql

CREATE OR REPLACE PROCEDURE generate_entity_group(
    p_variant_id           INTEGER,
    p_group_name           VARCHAR,
    p_entity_name_template VARCHAR,
    p_num_entities         INTEGER
)
LANGUAGE plpgsql AS $$
DECLARE
    v_entity_group_id  INTEGER;
    v_entity_id        INTEGER;
    v_entity_state_id  INTEGER;
    v_idx              INTEGER;
    v_entity_name      VARCHAR(255);
BEGIN
    INSERT INTO entity_group (name)
    VALUES (p_group_name)
    RETURNING id INTO v_entity_group_id;

    FOR v_idx IN 1..p_num_entities LOOP
        v_entity_name := format(p_entity_name_template, v_idx);

        INSERT INTO entity (variant_id, name)
        VALUES (p_variant_id, v_entity_name)
        RETURNING id INTO v_entity_id;

        INSERT INTO entity_group_link (entity_id, entity_group_id)
        VALUES (v_entity_id, v_entity_group_id);

        CALL generate_entity_state(v_entity_id, 0, NULL);
    END LOOP;
END;
$$;

Running query in 'postgresql://notebook@localhost:5432/sahuagin'

++
||
++
++

## Get Entity Group Details

In [814]:
%%sql

CREATE OR REPLACE FUNCTION get_entity_group_state_details(
    p_entity_group_id INT
)
RETURNS TEXT
LANGUAGE sql
AS $$
  SELECT jsonb_pretty(
           COALESCE(
             jsonb_object_agg(e.name, details),
             '{}'::jsonb
           )
         )
  FROM (
    SELECT e.name,
           (
             SELECT jsonb_agg(
                      jsonb_strip_nulls(
                        jsonb_build_object(
                          'Attribute', d."Attribute",
                          'Value', CASE 
                                     WHEN d."Value" IS NOT NULL 
                                       THEN CASE 
                                              WHEN d."Units" IS NOT NULL 
                                                THEN d."Value"::text || ' ' || d."Units"
                                              ELSE d."Value"::text
                                            END
                                     ELSE NULL
                                   END,
                          'Label', d."Label",
                          'Active Variations', CASE 
                                                 WHEN d."Active Variations" = '{}'::jsonb THEN NULL 
                                                 ELSE d."Active Variations" 
                                               END,
                          'Caused Variant', d."Caused Variant",
                          'Locked', CASE WHEN d."Locked" THEN d."Locked" ELSE NULL END
                        )
                      )
                    )
             FROM get_entity_state_details(e.id, 0) AS d
           ) AS details
    FROM entity e
    JOIN entity_group_link egl ON e.id = egl.entity_id
    WHERE egl.entity_group_id = p_entity_group_id
  ) e;
$$;

Running query in 'postgresql://notebook@localhost:5432/sahuagin'

++
||
++
++

## Set Span as Sub-Variant Activating Span

In [815]:
%%sql

CREATE OR REPLACE PROCEDURE set_variant_attr_span(
    p_variant_id INT,
    p_variant_attribute_id INT,
    p_span_id INT
)
LANGUAGE plpgsql
AS $$
BEGIN
  INSERT INTO variant_attr_span(variant_id, variant_attribute_id, span_id)
  VALUES (p_variant_id, p_variant_attribute_id, p_span_id);
END;
$$;

Running query in 'postgresql://notebook@localhost:5432/sahuagin'

++
||
++
++

## resolve_variant_path

In [816]:
%%sql

CREATE OR REPLACE FUNCTION resolve_variant_path(
    p_variant_id INT,
    p_path JSON
) RETURNS INT
LANGUAGE plpgsql
AS $$
DECLARE
    v_chain_length         INT;
    v_current_variant      INT := p_variant_id;
    v_parent_vavspan_attr  INT := NULL;
    v_i                    INT;
    v_step                 JSON;
    v_attr_name            TEXT;
    v_span_label           TEXT;
    v_variant_attribute_id INT;
    v_variant_attr_span_id INT;
    v_sub_variant          INT;
    v_vavspan_attr_id      INT;
BEGIN
    v_chain_length := json_array_length(p_path);
    IF v_chain_length = 0 THEN
       RAISE EXCEPTION 'Variant path is empty';
    END IF;
    
    FOR v_i IN 0 .. v_chain_length - 1 LOOP
        -- Extract the tuple [attribute name, span label] at index v_i.
        v_step := p_path->v_i;
        v_attr_name := v_step->>0;
        v_span_label := v_step->>1;
        
        -- Look up the variant_attribute by name in the current variant.
        SELECT id INTO v_variant_attribute_id
          FROM variant_attribute
         WHERE variant_id = v_current_variant
           AND name = v_attr_name
         LIMIT 1;
        IF v_variant_attribute_id IS NULL THEN
            RAISE EXCEPTION 'Variant attribute "%" not found in variant % at path index %',
                v_attr_name, v_current_variant, v_i;
        END IF;
        
        -- Look up the variant_attr_span record by joining with span using the span label.
        SELECT vas.id, vas.variant_id
          INTO v_variant_attr_span_id, v_sub_variant
          FROM variant_attr_span vas
          JOIN span s ON s.id = vas.span_id
         WHERE vas.variant_attribute_id = v_variant_attribute_id
           AND s.label = v_span_label
         LIMIT 1;
        IF v_variant_attr_span_id IS NULL THEN
            RAISE EXCEPTION 'No span with label "%" found for variant attribute "%" at path index %',
                v_span_label, v_attr_name, v_i;
        END IF;
        
        -- Resolve (or create) the unique vavspan_attr record for this step.
        SELECT id INTO v_vavspan_attr_id
          FROM vavspan_attr
         WHERE variant_attribute_id = v_variant_attribute_id
           AND variant_attr_span_id = v_variant_attr_span_id
           AND ((parent_vavspan_attr_id IS NULL AND v_parent_vavspan_attr IS NULL)
                OR parent_vavspan_attr_id = v_parent_vavspan_attr)
         LIMIT 1;
        IF v_vavspan_attr_id IS NULL THEN
            INSERT INTO vavspan_attr (variant_attribute_id, variant_attr_span_id, parent_vavspan_attr_id)
            VALUES (v_variant_attribute_id, v_variant_attr_span_id, v_parent_vavspan_attr)
            RETURNING id INTO v_vavspan_attr_id;
        END IF;
        
        -- If not at the final step, a sub-variant must be activated.
        IF v_i < v_chain_length - 1 THEN
            IF v_sub_variant IS NULL THEN
                RAISE EXCEPTION 'Chain impossible: attribute "%" with span label "%" at path index % does not activate a sub-variant',
                    v_attr_name, v_span_label, v_i;
            END IF;
            v_current_variant := v_sub_variant;
        END IF;
        
        -- Set up for the next iteration.
        v_parent_vavspan_attr := v_vavspan_attr_id;
    END LOOP;
    
    RETURN v_parent_vavspan_attr;
END;
$$;

Running query in 'postgresql://notebook@localhost:5432/sahuagin'

++
||
++
++

## get_vavspan_attr_chain

In [817]:
%%sql

CREATE OR REPLACE FUNCTION get_vavspan_attr_chain(
    p_vavspan_attr_id INT
) RETURNS TABLE(level INT, variant_attribute_id INT, causation_index INT)
LANGUAGE plpgsql
AS $$
BEGIN
RETURN QUERY WITH RECURSIVE chain(lev, vav_id, va_id, ci, parent) AS (
    -- Start at the leaf: assign level = 1.
    SELECT 1,
           v.id,
           v.variant_attribute_id,
           va.causation_index,
           v.parent_vavspan_attr_id
      FROM vavspan_attr v
      JOIN variant_attribute va ON va.id = v.variant_attribute_id
     WHERE v.id = p_vavspan_attr_id
    UNION ALL
    -- Walk upward.
    SELECT c.lev + 1,
           v.id,
           v.variant_attribute_id,
           va.causation_index,
           v.parent_vavspan_attr_id
      FROM vavspan_attr v
      JOIN variant_attribute va ON va.id = v.variant_attribute_id
      JOIN chain c ON v.id = c.parent
)
-- Reverse the order so that the top-most attribute is first.
SELECT row_number() OVER (ORDER BY lev DESC) AS level,
       va_id AS variant_attribute_id,
       ci AS causation_index
  FROM chain
 ORDER BY level;
END;
$$;

Running query in 'postgresql://notebook@localhost:5432/sahuagin'

++
||
++
++

## compare_vavspan_attr_order

In [818]:
%%sql

CREATE OR REPLACE FUNCTION compare_vavspan_attr_order(
    p_a INT,
    p_b INT
) RETURNS INT
LANGUAGE plpgsql
AS $$
DECLARE
    rec RECORD;
BEGIN
    FOR rec IN
        SELECT COALESCE(a.level, b.level) AS level,
               a.causation_index AS a_ci,
               b.causation_index AS b_ci
          FROM get_vavspan_attr_chain(p_a) a
          FULL OUTER JOIN get_vavspan_attr_chain(p_b) b USING (level)
         ORDER BY level
    LOOP
        -- If one chain is shorter, treat the missing value as "less".
        IF rec.a_ci IS NULL AND rec.b_ci IS NOT NULL THEN
            RETURN -1;
        ELSIF rec.b_ci IS NULL AND rec.a_ci IS NOT NULL THEN
            RETURN 1;
        ELSIF rec.a_ci < rec.b_ci THEN
            RETURN -1;
        ELSIF rec.a_ci > rec.b_ci THEN
            RETURN 1;
        END IF;
        -- Otherwise they are equal at this level; continue.
    END LOOP;
    RETURN 0;
END;
$$;

Running query in 'postgresql://notebook@localhost:5432/sahuagin'

++
||
++
++

## Add Variation

In [819]:
%%sql

CREATE OR REPLACE PROCEDURE add_variation(
    p_variant_id              INT,
    p_activating_variant_path JSON,
    p_to_modify_variant_path  JSON
)
LANGUAGE plpgsql
AS $$
DECLARE
    v_activating_vavspan_attr_id INT;
    v_to_modify_vavspan_attr_id  INT;
    cmp_result INT;
BEGIN
    -- Resolve each branch (using the previously defined resolve_variant_path).
    v_activating_vavspan_attr_id := resolve_variant_path(p_variant_id, p_activating_variant_path);
    v_to_modify_vavspan_attr_id  := resolve_variant_path(p_variant_id, p_to_modify_variant_path);
    
    -- Compare the two chains.
    cmp_result := compare_vavspan_attr_order(v_activating_vavspan_attr_id, v_to_modify_vavspan_attr_id);
    IF cmp_result <> -1 THEN
        RAISE EXCEPTION 'Invalid variation ordering: activating branch (vavspan_attr id %) is not generated before to-modify branch (vavspan_attr id %); comparison result = %',
            v_activating_vavspan_attr_id, v_to_modify_vavspan_attr_id, cmp_result;
    END IF;
    
    -- If the order is acceptable, insert the new variation.
    INSERT INTO variation(
        to_modify_vavspan_attr_id,
        activating_vavspan_attr_id
    )
    VALUES (
        v_to_modify_vavspan_attr_id,
        v_activating_vavspan_attr_id
    );
END;
$$;

Running query in 'postgresql://notebook@localhost:5432/sahuagin'

++
||
++
++

## get_variant_id

In [820]:
%%sql

CREATE OR REPLACE FUNCTION get_variant_id(
    p_variant_name VARCHAR
) RETURNS INTEGER
LANGUAGE plpgsql
AS $$
DECLARE
    variant_id INTEGER;
BEGIN
    SELECT id
      INTO variant_id
      FROM variant
     WHERE name = p_variant_name;
     
    IF NOT FOUND THEN
        RAISE EXCEPTION 'Variant "%" not found', variant_id;
    END IF;
    
    RETURN variant_id;
END;
$$;

Running query in 'postgresql://notebook@localhost:5432/sahuagin'

++
||
++
++

## get_attribute_id

In [821]:
%%sql

CREATE OR REPLACE FUNCTION get_attribute_id(
    p_attribute_name VARCHAR
) RETURNS INTEGER
LANGUAGE plpgsql
AS $$
DECLARE
    attribute_id INTEGER;
BEGIN
    SELECT id
      INTO attribute_id
      FROM attribute
     WHERE name = p_attribute_name;
     
    IF NOT FOUND THEN
        RAISE EXCEPTION 'Attribute "%" not found', p_attribute_name;
    END IF;
    
    RETURN attribute_id;
END;
$$;

Running query in 'postgresql://notebook@localhost:5432/sahuagin'

++
||
++
++

## get_span_id

In [822]:
%%sql

CREATE OR REPLACE FUNCTION get_span_id(
    p_attribute_name VARCHAR,
    p_span_type      attribute_type,
    p_span_label     VARCHAR
) RETURNS INTEGER
LANGUAGE plpgsql
AS $$
DECLARE
    span_id INTEGER;
BEGIN
    SELECT s.id
      INTO span_id
      FROM span s
      JOIN attribute a ON a.id = s.attribute_id
     WHERE s.label = p_span_label
       AND a.name = p_attribute_name
       AND s.type = p_span_type;

    IF NOT FOUND THEN
        RAISE EXCEPTION 'Span on attribute "%" of type "%" and label "%" not found', p_attribute_name, p_span_type, p_span_label;
    END IF;

    RETURN span_id;
END;
$$;

Running query in 'postgresql://notebook@localhost:5432/sahuagin'

++
||
++
++

## get_variation_id_by_paths

In [823]:
%%sql

CREATE OR REPLACE FUNCTION get_variation_id_by_paths(
    p_variant_id              INT,
    p_activating_variant_path JSON,
    p_to_modify_variant_path  JSON
) RETURNS INT
LANGUAGE plpgsql
AS $$
DECLARE
    v_activating_vavspan_attr INT;
    v_to_modify_vavspan_attr  INT;
    v_variation_id          INT;
BEGIN
    -- Resolve the activating branch to get its terminal vavspan_attr id.
    v_activating_vavspan_attr := resolve_variant_path(p_variant_id, p_activating_variant_path);
    
    -- Resolve the to–modify branch similarly.
    v_to_modify_vavspan_attr := resolve_variant_path(p_variant_id, p_to_modify_variant_path);
    
    -- Try to find an existing variation with these branch endpoints.
    SELECT id INTO v_variation_id
      FROM variation
     WHERE variant_id = p_variant_id
       AND activating_vavspan_attr_id = v_activating_vavspan_attr
       AND to_modify_vavspan_attr_id = v_to_modify_vavspan_attr
     LIMIT 1;
    
    -- If no such variation exists, create it.
    IF v_variation_id IS NULL THEN
        INSERT INTO variation(
            variant_id,
            activating_vavspan_attr_id,
            to_modify_vavspan_attr_id
        )
        VALUES (
            p_variant_id,
            v_activating_vavspan_attr,
            v_to_modify_vavspan_attr
        )
        RETURNING id INTO v_variation_id;
    END IF;
    
    RETURN v_variation_id;
END;
$$;

Running query in 'postgresql://notebook@localhost:5432/sahuagin'

++
||
++
++

## get_effect_id

In [824]:
%%sql

CREATE OR REPLACE FUNCTION get_effect_id(
    p_variant_name   VARCHAR,
    p_effect_name VARCHAR
) RETURNS INTEGER
LANGUAGE plpgsql
AS $$
DECLARE
    effect_id INTEGER;
BEGIN
    SELECT e.id
      INTO effect_id
      FROM effect e
      JOIN variant vt ON vt.id = e.variant_id
     WHERE vt.name = p_variant_name
       AND e.name = p_effect_name;

    IF NOT FOUND THEN
        RAISE EXCEPTION 'effect "%" in variant "%" not found', p_effect_name, p_variant_name;
    END IF;

    RETURN effect_id;
END;
$$;

Running query in 'postgresql://notebook@localhost:5432/sahuagin'

++
||
++
++

## Add Parameter Delta Effect to Variation

In [825]:
%%sql

CREATE OR REPLACE PROCEDURE set_continuous_variation_span(
    p_name                    VARCHAR,
    p_variant_name            VARCHAR,
    p_activating_variant_path JSON,
    p_to_modify_variant_path  JSON,
    p_delta_mode              DOUBLE PRECISION,
    p_delta_conc              DOUBLE PRECISION,
    p_delta_skew              DOUBLE PRECISION
)
LANGUAGE plpgsql
AS $$
DECLARE
    v_variant_id   INT;
    v_variation_id INT;
    v_effect_id    INT;
BEGIN
    v_variant_id := get_variant_id(p_variant_name);
    v_variation_id := get_variation_id_by_paths(v_variant_id, p_activating_variant_path, p_to_modify_variant_path);
    
    INSERT INTO effect(name, variant_id, variation_id)
    VALUES (p_name, v_variant_id, v_variation_id)
    ON CONFLICT (name, variant_id) DO UPDATE
       SET variation_id = EXCLUDED.variation_id
    RETURNING id INTO v_effect_id;
    
    INSERT INTO variation_continuous_attr(effect_id, delta_mode, delta_conc, delta_skew)
    VALUES (v_effect_id, p_delta_mode, p_delta_conc, p_delta_skew);
END;
$$;

Running query in 'postgresql://notebook@localhost:5432/sahuagin'

++
||
++
++

## Add Additional Possibility Effect to Variation

In [826]:
%%sql

CREATE OR REPLACE PROCEDURE set_span_activating_span(
    p_name                    VARCHAR,
    p_variant_name            VARCHAR,
    p_activating_variant_path JSON,
    p_to_modify_variant_path  JSON,
    p_span_label              VARCHAR,
    p_weight                  DOUBLE PRECISION
)
LANGUAGE plpgsql
AS $$
DECLARE
    v_variant_id   INT;
    v_variation_id INT;
    v_attribute_id INT;
    v_span_id      INT;
    v_effect_id    INT;
BEGIN
    v_variant_id := get_variant_id(p_variant_name);
    v_variation_id := get_variation_id_by_paths(v_variant_id, p_activating_variant_path, p_to_modify_variant_path);

    INSERT INTO effect(name, variant_id, variation_id)
    VALUES (p_name, v_variant_id, v_variation_id)
    ON CONFLICT (name, variant_id) DO UPDATE
       SET variation_id = EXCLUDED.variation_id
    RETURNING id INTO v_effect_id;

    SELECT a.id
      INTO v_attribute_id
      FROM variation var
      JOIN vavspan_attr vav ON var.to_modify_vavspan_attr_id = vav.id
      JOIN variant_attribute va ON va.id = vav.variant_attribute_id
      JOIN attribute a ON a.id = va.attribute_id
     WHERE var.id = v_variation_id;

    INSERT INTO span(attribute_id, label, type, is_percentage_pinned, weight, max_value, min_value)
    VALUES (v_attribute_id, p_span_label, 'discrete', false, p_weight, NULL, NULL)
    RETURNING id INTO v_span_id;
    
    INSERT INTO variation_activated_span(variation_id, span_id)
    VALUES (v_variation_id, v_span_id);
END;
$$;

Running query in 'postgresql://notebook@localhost:5432/sahuagin'

++
||
++
++

### delete_orphaned_span

In [827]:
%%sql

CREATE OR REPLACE FUNCTION delete_orphaned_span() RETURNS TRIGGER AS $$
BEGIN
    DELETE FROM span WHERE id = OLD.span_id;
    RETURN OLD;
END;
$$ LANGUAGE plpgsql;

Running query in 'postgresql://notebook@localhost:5432/sahuagin'

++
||
++
++

### trg_delete_span_after_activation_delete

In [828]:
%%sql

CREATE OR REPLACE TRIGGER trg_delete_span_after_activation_delete
AFTER DELETE ON variation_activated_span
FOR EACH ROW
EXECUTE FUNCTION delete_orphaned_span();

Running query in 'postgresql://notebook@localhost:5432/sahuagin'

++
||
++
++

## Add Delta Weight Effect to Variation

In [829]:
%%sql

CREATE OR REPLACE PROCEDURE set_delta_weight_span(
    p_name                    VARCHAR,
    p_variant_name            VARCHAR,
    p_activating_variant_path JSON,
    p_to_modify_variant_path  JSON,
    p_span_label              VARCHAR,
    p_delta_weight            DOUBLE PRECISION
)
LANGUAGE plpgsql
AS $$
DECLARE
    v_variant_id   INT;
    v_variation_id INT;
    v_attribute_id INT;
    v_span_id      INT;
    v_effect_id    INT;
BEGIN
    v_variant_id := get_variant_id(p_variant_name);
    v_variation_id := get_variation_id_by_paths(v_variant_id, p_activating_variant_path, p_to_modify_variant_path);

    INSERT INTO effect(name, variant_id, variation_id)
    VALUES (p_name, v_variant_id, v_variation_id)
    ON CONFLICT (name, variant_id) DO UPDATE
       SET variation_id = EXCLUDED.variation_id
    RETURNING id INTO v_effect_id;

    SELECT a.id
      INTO v_attribute_id
      FROM variation var
      JOIN vavspan_attr vav ON var.to_modify_vavspan_attr_id = vav.id
      JOIN variant_attribute va ON va.id = vav.variant_attribute_id
      JOIN attribute a ON a.id = va.attribute_id
     WHERE var.id = v_variation_id;

    v_span_id := get_span_id(p_attribute_name, 'discrete', p_span_label);
    
    INSERT INTO variation_delta_weight(variation_id, span_id, delta_weight)
    VALUES (v_variation_id, v_span_id, p_delta_weight);
END;
$$;

Running query in 'postgresql://notebook@localhost:5432/sahuagin'

++
||
++
++

## Add Removed Possibility Effect to Variation

In [830]:
%%sql

CREATE OR REPLACE PROCEDURE add_removed_possibility_effect(
    p_name                    VARCHAR,
    p_variant_name            VARCHAR,
    p_activating_variant_path JSON,
    p_to_modify_variant_path  JSON,
    p_span_label              VARCHAR
)
LANGUAGE plpgsql
AS $$
DECLARE
    v_variant_id   INT;
    v_variation_id INT;
    v_attribute_id INT;
    v_span_id      INT;
    v_effect_id    INT;
BEGIN
    v_variant_id := get_variant_id(p_variant_name);
    v_variation_id := get_variation_id_by_paths(v_variant_id, p_activating_variant_path, p_to_modify_variant_path);

    INSERT INTO effect(name, variant_id, variation_id)
    VALUES (p_name, v_variant_id, v_variation_id)
    ON CONFLICT (name, variant_id) DO UPDATE
       SET variation_id = EXCLUDED.variation_id
    RETURNING id INTO v_effect_id;

    SELECT a.id
      INTO v_attribute_id
      FROM variation var
      JOIN vavspan_attr vav ON var.to_modify_vavspan_attr_id = vav.id
      JOIN variant_attribute va ON va.id = vav.variant_attribute_id
      JOIN attribute a ON a.id = va.attribute_id
     WHERE var.id = v_variation_id;

    v_span_id := get_span_id(p_attribute_name, 'discrete', p_span_label);
    
    INSERT INTO variation_inactive_span(variation_id, span_id)
    VALUES (v_variation_id, v_span_id);
END;
$$;

Running query in 'postgresql://notebook@localhost:5432/sahuagin'

++
||
++
++

# Inspect Schema

## List Tables

In [831]:
%%sql

SELECT tablename AS name
FROM pg_catalog.pg_tables
WHERE schemaname NOT IN ('pg_catalog', 'information_schema')
ORDER BY schemaname, tablename;

Running query in 'postgresql://notebook@localhost:5432/sahuagin'

19 rows affected.

name
attribute
debug_log
effect
entity
entity_group
entity_group_link
entity_state
entity_varattr_value
evav_lock
span


## List Procedures

In [832]:
%%sql

SELECT proname AS name, pg_catalog.pg_get_function_arguments(p.oid) AS arguments
FROM pg_catalog.pg_proc p
JOIN pg_catalog.pg_namespace n ON n.oid = p.pronamespace
WHERE p.prokind = 'p' AND n.nspname = 'public'
ORDER BY n.nspname, proname;

Running query in 'postgresql://notebook@localhost:5432/sahuagin'

30 rows affected.

name,arguments
add_cont_span_to_attr,"IN p_attribute_id integer, IN p_label character varying, IN p_new_min double precision, IN p_new_max double precision"
add_continuous_attribute,"IN p_name character varying, IN p_min_value double precision, IN p_mode_value double precision, IN p_max_value double precision, IN p_concentration double precision, IN p_skew double precision, IN p_decimals integer, IN p_has_labels boolean, IN p_has_value boolean, IN p_units character varying, IN p_spans json DEFAULT NULL::json"
add_continuous_attribute,"IN p_name character varying, IN p_min_value double precision, IN p_mode_value double precision, IN p_max_value double precision, IN p_concentration double precision, IN p_skew double precision, IN p_decimals integer, IN p_has_labels boolean, IN p_has_value boolean, IN p_units character varying"
add_disc_span_to_attr,"IN p_attribute_id integer, IN p_label character varying"
add_discrete_attribute,IN in_name character varying
add_discrete_attribute,"IN in_name character varying, IN in_spans json"
add_entity,"IN in_name character varying, IN in_variant_id integer"
add_removed_possibility_effect,"IN p_name character varying, IN p_variant_name character varying, IN p_activating_variant_path json, IN p_to_modify_variant_path json, IN p_span_label character varying"
add_variant,"IN in_variant_name character varying, IN in_attr_keys json"
add_variant,IN in_variant_name character varying


## List Functions

In [833]:
%%sql

SELECT proname AS name,
       pg_catalog.pg_get_function_arguments(p.oid) AS arguments,
       pg_catalog.pg_get_function_result(p.oid) AS return_type
FROM pg_catalog.pg_proc p
JOIN pg_catalog.pg_namespace n ON n.oid = p.pronamespace
WHERE p.prokind = 'f' AND n.nspname = 'public'
ORDER BY n.nspname, proname;

Running query in 'postgresql://notebook@localhost:5432/sahuagin'

20 rows affected.

name,arguments,return_type
compare_vavspan_attr_order,"p_a integer, p_b integer",integer
debug_log,"p_procedure_name character varying, p_log_message text",void
delete_orphaned_span,,trigger
gamma_rng,shape double precision,double precision
get_attribute_id,p_attribute_name character varying,integer
get_continuous_attribute,p_attribute_name character varying,"TABLE(name character varying, type attribute_type, min double precision, mode double precision, max double precision, concentration double precision, skew double precision, units character varying, decimals integer, has_value boolean, has_labels boolean, span_label character varying, span_min double precision, span_max double precision)"
get_discrete_attribute,p_attribute_name character varying,"TABLE(attribute character varying, type attribute_type, span_label character varying, span_weight double precision, is_pinned boolean)"
get_effect_id,"p_variant_name character varying, p_effect_name character varying",integer
get_entity_group_state_details,p_entity_group_id integer,text
get_entity_state_details,"p_entity_id integer, p_time double precision",SETOF entity_state_details


## List Indices

In [834]:
%%sql

SELECT i.relname AS index_name, t.relname AS table_name
FROM pg_catalog.pg_index ix
JOIN pg_catalog.pg_class i ON i.oid = ix.indexrelid
JOIN pg_catalog.pg_class t ON t.oid = ix.indrelid
JOIN pg_catalog.pg_namespace n ON n.oid = t.relnamespace
WHERE n.nspname = 'public'
ORDER BY n.nspname, t.relname, i.relname;

Running query in 'postgresql://notebook@localhost:5432/sahuagin'

73 rows affected.

index_name,table_name
attribute_name_key,attribute
attribute_pkey,attribute
idx_attribute_name,attribute
debug_log_pkey,debug_log
effect_pkey,effect
unique_effect_name_within_variant,effect
entity_name_key,entity
entity_pkey,entity
entity_group_name_key,entity_group
entity_group_pkey,entity_group


# Testing

## Old

In [ ]:
%%sql

CREATE OR REPLACE FUNCTION run_test_1()
  RETURNS TEXT
AS $$
DECLARE
  race        integer;
  gender      integer;
  male        integer;
  female      integer;
  hair_color  integer;
  blonde      integer;
  brown       integer;
  golden      integer;
  auburn      integer;
  ginger      integer;
  sandy       integer;
  black       integer;
  gray        integer;
  height      integer;
  short       integer;
  average     integer;
  tall        integer;
  human       integer;
  human_gender  integer;
  human_hair_color  integer;
  human_height  integer;
  human_red_hair_to_height_var  integer;
  test_humans integer;
BEGIN
  CALL add_discrete_attribute('Gender', gender);
  CALL add_disc_span_to_attr(gender, 'Male', male);
  CALL add_disc_span_to_attr(gender, 'Female', female);

  CALL add_discrete_attribute('Hair Color', hair_color);
  CALL add_disc_span_to_attr(hair_color, 'Blonde', blonde);
  CALL add_disc_span_to_attr(hair_color, 'Brown', brown);
  CALL add_disc_span_to_attr(hair_color, 'Golden', golden);
  CALL add_disc_span_to_attr(hair_color, 'Auburn', auburn);
  CALL add_disc_span_to_attr(hair_color, 'Ginger', ginger);
  CALL add_disc_span_to_attr(hair_color, 'Sandy', sandy);
  CALL add_disc_span_to_attr(hair_color, 'Black', black);
  CALL add_disc_span_to_attr(hair_color, 'Gray', gray);
  CALL modify_disc_span_weight(brown, .5);
  CALL modify_disc_span_weight(gray, .05);

  -- add_continuous_attribute(name, min, mode, max, concentration, skew, precision, show_spans, show_value, units, out p_attr_id)
  CALL add_continuous_attribute('Height', 0, 222, 1000, 100000, 0, 4, TRUE, TRUE, 'in.', height);
  CALL add_cont_span_to_attr(height, 'Short', 0, 100, short);
  CALL add_cont_span_to_attr(height, 'Average', 100, 400, average);
  CALL add_cont_span_to_attr(height, 'Tall', 400, 1000, tall);

  CALL add_variant('Human', human);
  CALL link_attr_to_variant(human, gender, 'Gender', NULL, human_gender);
  CALL link_attr_to_variant(human, hair_color, 'Hair Color', NULL, human_hair_color);
  CALL link_attr_to_variant(human, height, 'Height', NULL, human_height);

  CALL add_variation(ginger, NULL, human_hair_color, NULL, human_height, human_red_hair_to_height_var);

  CALL generate_entity_group(human, 'Test Humans', 'Human %s', 100, test_humans);
  RETURN get_entity_group_state_details(test_humans);
END;
$$ LANGUAGE plpgsql;

SELECT * FROM run_test_1()

## Gender Attribute

### Delete Gender Attribute

In [842]:
%%sql

CALL delete_discrete_attribute('Gender');

Running query in 'postgresql://notebook@localhost:5432/sahuagin'

++
||
++
++

### Add Gender Attribute

In [843]:
%%sql

call add_discrete_attribute('Gender', '[
    ["Male", 1], 
    ["Female", 1]
]');

Running query in 'postgresql://notebook@localhost:5432/sahuagin'

++
||
++
++

### View Gender Attribute

In [844]:
%%sql

SELECT * FROM get_discrete_attribute('Gender');

Running query in 'postgresql://notebook@localhost:5432/sahuagin'

2 rows affected.

attribute,type,span_label,span_weight,is_pinned
Gender,discrete,Female,0.5,False
Gender,discrete,Male,0.5,False


## Hair Color Attribute

### Delete Hair Color Attribute

In [845]:
%%sql

CALL delete_discrete_attribute('Hair Color');

Running query in 'postgresql://notebook@localhost:5432/sahuagin'

++
||
++
++

### Add Hair Color Attribute

In [846]:
%%sql

call add_discrete_attribute('Hair Color', '[
    ["Blonde", 2], 
    ["Brown", 3],
    ["Golden", 2],
    ["Auburn", 2],
    ["Ginger", 1],
    ["Sandy", 2],
    ["Black", 3],
    ["Gray", 2]
]');

Running query in 'postgresql://notebook@localhost:5432/sahuagin'

++
||
++
++

### View Hair Color Attribute

In [847]:
%%sql

SELECT * FROM get_discrete_attribute('Hair Color');

Running query in 'postgresql://notebook@localhost:5432/sahuagin'

8 rows affected.

attribute,type,span_label,span_weight,is_pinned
Hair Color,discrete,Ginger,0.058823529411764705,False
Hair Color,discrete,Auburn,0.11764705882352941,False
Hair Color,discrete,Blonde,0.11764705882352941,False
Hair Color,discrete,Golden,0.11764705882352941,False
Hair Color,discrete,Gray,0.11764705882352941,False
Hair Color,discrete,Sandy,0.11764705882352941,False
Hair Color,discrete,Black,0.17647058823529413,False
Hair Color,discrete,Brown,0.17647058823529413,False


## Height Attribute

### Delete Height Attribute

In [848]:
%%sql

CALL delete_continuous_attribute('Height');

Running query in 'postgresql://notebook@localhost:5432/sahuagin'

++
||
++
++

### Add Height Attribute

In [849]:
%%sql

call add_continuous_attribute('Height', 0, 222, 1000, 100000, 0, 4, true, true, 'in.', '[
    ["Short", 100],
    ["Average", 400],
    ["Tall", 1000]
]');

Running query in 'postgresql://notebook@localhost:5432/sahuagin'

++
||
++
++

### View Height Attribute

In [850]:
%%sql

SELECT * FROM get_continuous_attribute('Height');

Running query in 'postgresql://notebook@localhost:5432/sahuagin'

3 rows affected.

name,type,min,mode,max,concentration,skew,units,decimals,has_value,has_labels,span_label,span_min,span_max
Height,continuous,0.0,222.0,1000.0,100000.0,0.0,in.,4,True,True,Average,100.0,400.0
Height,continuous,0.0,222.0,1000.0,100000.0,0.0,in.,4,True,True,Short,0.0,100.0
Height,continuous,0.0,222.0,1000.0,100000.0,0.0,in.,4,True,True,Tall,400.0,1000.0


## Human Variant

### Delete

In [851]:
%%sql

CALL delete_variant('Human');

Running query in 'postgresql://notebook@localhost:5432/sahuagin'

++
||
++
++

### Add

In [857]:
%%sql

CALL add_variant('Human', '[
    "Gender",
    "Hair Color",
    "Height"
]');

Running query in 'postgresql://notebook@localhost:5432/sahuagin'

++
||
++
++

### Get

In [882]:
%%sql

SELECT * FROM get_variant_page('Human', 1, 1000);

Running query in 'postgresql://notebook@localhost:5432/sahuagin'

3 rows affected.

overall_index,label,to_modify_effects,activated_effects
1,Gender,[],[]
2,Hair Color,[],[]
3,Height,[],[]
